# RAG Project

LangChain Tutorials: https://python.langchain.com/docs/tutorials/

LangChain HowTos: https://python.langchain.com/docs/how_to/

LangChain Conceptual Guide: https://python.langchain.com/docs/concepts/#retrieval

High-Level Overview of RAG: https://python.langchain.com/docs/tutorials/rag/



In [ ]:
%pip install --quiet --upgrade bitsandbytes langchain langchain-community langchain-huggingface transformers beautifulsoup4 faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 100.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.

## **Part 1: Setup LLM**

**Set up LLM**
- https://python.langchain.com/docs/integrations/providers/huggingface/

<br/>
<br/>

<u>**Experimentation and Improvements** (not very important)</u>
- Find a better LLM that also fits into memory
  - Mistral-Small-Instruct-2409: Positioned Between Mistral NeMo 12B and Mistral Large 123B (crashed)
    - https://www.marktechpost.com/2024/09/18/mistral-ai-released-mistral-small-instruct-2409-a-game-changing-open-source-language-model-empowering-versatile-ai-applications-with-unmatched-efficiency-and-accessibility/
    - https://huggingface.co/mistralai/Mistral-Small-Instruct-2409
  - Mistral Nemo (Mistral-Nemo-Instruct-2407) (12 mins with quantization)
      - https://www.reddit.com/r/LocalLLaMA/comments/1eg5j2t/which_small_model_12b_do_you_guys_are_using_for/
      - https://huggingface.co/mistralai/Mistral-Nemo-Instruct-2407
  - Mistral 7B Quantized, Mistral 7B Instruct
    - https://www.reddit.com/r/LocalLLaMA/comments/1av779p/experiences_with_smaller_models_with_rag/

  - Qwen2–72B-Instruct, Qwen1.5-32B-Chat
      - https://medium.com/@naman1011/whats-the-best-llm-to-use-for-rag-476bec1bfa97
      - https://huggingface.co/Qwen/Qwen2.5-72B-Instruct

Quantization
- https://medium.com/@rakeshrajpurohit/model-quantization-with-hugging-face-transformers-and-bitsandbytes-integration-b4c9983e8996

In [ ]:
import torch
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import BitsAndBytesConfig

#from huggingface_hub import notebook_login
#notebook_login()

# We load the quantized weights for faster generation
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

model_id = "mistralai/Mistral-Nemo-Instruct-2407"
# model_id = "Qwen/Qwen2.5-0.5B-Instruct" # choose this for faster inference
tokenizer = AutoTokenizer.from_pretrained(model_id)
#model = AutoModelForCausalLM.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,quantization_config=quantization_config)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer,max_new_tokens=1000)
llm = HuggingFacePipeline(pipeline=pipe)

tokenizer_config.json:   0%|          | 0.00/181k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.26M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


...

model-00003-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

## **Part 2: Load Document**

Load Data
- https://python.langchain.com/docs/how_to/#document-loaders

Can try loading data from web pages using the <em>Simple and Fast Parsing Approach</em> to recover one ```Document``` object per webpage
- https://python.langchain.com/docs/how_to/document_loader_web/
- https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.web_base.WebBaseLoader.html
- https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html

Going to use text document because the web one had too much irrelevant info
- https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.text.TextLoader.html

<br/>
<br/>

<u>Concern for Data collection</u>

- Example: When we ask the model what are the best places for hiking, and we have 10+ documents for hiking destinations, will that be a problem? Maybe we need metadata to denote the location
- Example: When we have 1 document that is all about hiking like 20 best hikes, how much is to be retrieved? -> Experiment with chunk and overlap size, parent-document retrieval approaches

<br/>
<br/>

<u>**Experimentation and Improvements**</u>

<u>1. Decide the categories of documents that we are going to put in (by country) and how many of each</u>
- Activities
- Food
- Accomodation
- Transport
- Places of interest
- etc.

<u>2. Scrape the web better</u>
- Use advanced parsing
- Use lazy loading and async for efficiency
- Specify parameters for BeautifulSoup through bs_kwargs to pick up only body text instead of extra info like navigation bars i.e. Parse web pages better

<u>3. Instead of scraping web, use other sources such as text documents</u>
- Find another way to get data

<u>4. Organize data (either local folder of text documents or through cloud or DB?)</u>

In [ ]:
"""
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_path = "https://www.earthtrekkers.com/norway-bucket-list-best-things-to-do-in-norway/"
)

docs = loader.load()
"""

from langchain_community.document_loaders import TextLoader
# Change to reading by a directory with all our text files if we are really going ahead with reading from text files
loader = TextLoader(
    file_path = "Norway Example.txt"
)

docs = loader.load()

In [ ]:
docs

[Document(metadata={'source': 'Norway Example.txt'}, page_content='15 of the Most Beautiful Fjord Hikes in Norway\n\nGlacial lagoons, verdant forests and soaring mountains - our guide to the best fjord hikes in Norway...\n\nLooking for the best fjord hikes in Norway? Hiking is second nature to Norwegians and with the vast and varied landscapes on their doorstep, it’s easy to see why. From the white-sand beaches of the Lofoten Islands to the dramatic mountains of the south, and the spectacular fjords in between, there’s something to suit all types of hiker. It\'s Norway\'s fjords we\'re going to focusing on today - because many of the best hiking in Norway happens to be along, around or above a fjord.\n\nMany fjords promise spectacular natural views of untouched verdant forests, ice cold lagoons and sweeping views over seemingly endless cliffs and mountains. They\'re also great places to chase the Northern Lights during winter. Here, we\'re going to give you an introduction to the best 

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

## **Part 3: Chunking/Split Text**

Split LangChain ```Document``` objects into smaller chunks. This is useful for
- Indexing and retrieving relevant data
- LLM (cannot fit into model's finite context window)
- Overcoming embedding model size limitations
- https://python.langchain.com/docs/concepts/text_splitters/
- https://python.langchain.com/docs/how_to/#text-splitters

Try ```RecursiveCharacterTextSplitter``` first
- Overlap helps to mitigate the possibility of separating a statement from important context related to it
- ```RecursiveCharacterTextSplitter``` recursively split the document using common separators like new lines until each chunk is the appropriate size. This is the recommended text splitter for generic text use cases
- Set ```add_start_index=True``` so that the character index at which each split Document starts within the initial Document is preserved as metadata attribute ```“start_index”```.
- This is a text-structured based splitting approach: Creating split that maintain natural language flow, maintain semantic coherence within split, and adapts to varying levels of text granularity

<br/>
<br/>

<u>**Experimentation and Improvements**</u>
1. Clean Data
2. Add metadata to chunks
  - This is for self query: https://python.langchain.com/docs/how_to/self_query/
  - I think some useful metadata that could be useful could be activity type and country. Not sure about other metadata
3. Experiment with Chunk size and Overlap size for recursive text splitting
4. Experiment with other splitting strategies: Document-structured based splitting, Semantic meaning based splitting (not sure if relevant)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True # Following tutorial specs
)
all_splits = text_splitter.split_documents(docs)

In [ ]:
all_splits

[Document(metadata={'source': 'Norway Example.txt', 'start_index': 0}, page_content="15 of the Most Beautiful Fjord Hikes in Norway\n\nGlacial lagoons, verdant forests and soaring mountains - our guide to the best fjord hikes in Norway...\n\nLooking for the best fjord hikes in Norway? Hiking is second nature to Norwegians and with the vast and varied landscapes on their doorstep, it’s easy to see why. From the white-sand beaches of the Lofoten Islands to the dramatic mountains of the south, and the spectacular fjords in between, there’s something to suit all types of hiker. It's Norway's fjords we're going to focusing on today - because many of the best hiking in Norway happens to be along, around or above a fjord.\n\nMany fjords promise spectacular natural views of untouched verdant forests, ice cold lagoons and sweeping views over seemingly endless cliffs and mountains. They're also great places to chase the Northern Lights during winter. Here, we're going to give you an introduction

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

## **Part 4: Indexing/Storing chunks/splits**

Embed the contents of each chunk/split and insert these embeddings into a vector store
- When we want to search over our splits, we take a text search query, embed it, and perform some sort of “similarity” search to identify the stored splits with the most similar embeddings to our query embedding.
- Simplest similarity measure is cosine similarity. We measure the cosine of the angle between each pair of embeddings (which are high dimensional vectors).
- https://python.langchain.com/docs/how_to/
- https://python.langchain.com/docs/how_to/embed_text/
- https://python.langchain.com/docs/concepts/embedding_models/
- https://python.langchain.com/docs/how_to/vectorstores/
- https://python.langchain.com/docs/concepts/vectorstores/
- https://python.langchain.com/docs/integrations/vectorstores/
  - https://python.langchain.com/docs/integrations/vectorstores/faiss/
  - https://python.langchain.com/api_reference/community/vectorstores/langchain_community.vectorstores.faiss.FAISS.html

Embedding vectors (capture semantic meanings) can be compared easily
- Use ```embed_documents``` to embed multiple texts (documents)
- Use ```embed_query``` to embed a single text (query)
- As they are a set of coords in high-dim space, their similarity can be measured by
  - Cosine Similarity, Euclidean Distance, Dot Product

Vector Store
- Takes care of storing embedding vectors and performing similarity vector search against embedded query, therefore helping to retrieve relevant information based on semantic similarity
- Standard interface for working with vector stores: ```add_documents```, ```delete_documents```, ```similarity_search```
- Choice of similarity metric can sometimes be selected when initlaising vector store: refer to documentation of specific vectorstore we are using
- Choice of similarity search: Given the similarity metric to measure distance between the embedded query and embedded chunks, we need an algo to efficiently search over all embedded chunks. Many vector stores implement ```Hierarchical Navigable Small World```, a graph-based index structure that allows for eficient similarity search. See what search parameters we can add for specific choice of vector store
- Many vector stores support metadata filtering: Allows structured filters to reduce size of similarity search space
- Semantic search and Metadata search work hand in hand

<br/>

For now, use Hugging Face's ```sentence-transformers/all-mpnet-base-v2``` embedding model and ```FAISS``` as the vector store

<br/>
<br/>

<u>**Experimentation and Improvements**</u>

1. Explore other indexes used by FAISS
  - In our HW we played with Nbits for LSH
  - Compared cosine similarity, search time, precision, recall when changing Nbits
2. Explore Hierarchical Navigable Small World (HNSW), knowledge graphs
3. Explore other vector DBs like Chroma, Spotify's Annoy (not a priority unless their search methods are novel)
4. Explore other embedding models (not a priority)
5. IR System Evaluation Metrics (Must do)
  - Binary Relevance
  - Multiple Levels of Relevance

Note: Comparison of FAISS with chroma
- https://medium.com/@stepkurniawan/comparing-faiss-with-chroma-vector-stores-0953e1e619eb




In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore

embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

index = faiss.IndexFlatL2(len(embeddings_model.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings_model,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

vector_store.add_documents(all_splits)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

['d49596f6-8f31-4f21-b055-f38fba0cbbd4',
 '68d44cf3-f790-4ce4-9da4-f7d420e15da7',
 'c22be216-987d-4b46-b1a0-224ef063156b',
 'b9f94fad-72f0-4bcb-807b-364f404deaac',
 'c3f037c7-0a10-4fae-99ab-5a2c96880d5a',
 'c296da7b-157e-44aa-b29d-0f166ea04ab2',
 '6bf9b5f7-e568-41eb-af4a-951332232cde',
 '3eab7335-16c9-4be6-a00e-11a8932a026c',
 '8e26dbec-e58c-4c66-a4c6-b38ec228b2e3',
 'f9dccba1-08a5-4ae5-b75d-7421e16b1b7b',
 '52f818b5-6b49-41c6-b737-77a2031c3231',
 'dee565f9-c45c-4e7b-9b81-d75332dcc55c',
 'b1eabe8f-5319-4ae9-8411-ef41c20bef7e',
 'f73fa1f5-51ea-4d77-84c1-31c8fc815226',
 '6a269c5c-1bb7-4736-b0ce-9399cf24b938',
 'c7099369-22b7-4996-89ab-48e66bafb3b9',
 '09df9862-3468-4593-abc5-58ff7415d799',
 'fd5f31b7-2c34-4ace-8362-83060b44a313',
 'bb409ba4-f6c3-4517-8069-ba6c0ed48cbc',
 '91fbcdb2-7a55-44b2-94f0-a69bb8da3e4c',
 'b7fa1077-c404-4b75-b57d-d5f7cc8d891d',
 'e3fbae0c-d614-40cf-9f2f-d502b081a2ba',
 'f0fc0910-fe27-48e9-a6db-9043a86ad154',
 '85befd53-76af-417a-a8ca-c09715a23b78',
 'c4dd5de8-77ae-

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

## **Part 5: Retrieval**

Given a user query, retrieve relevant splits from our vector store
- Experimentations (See experimentation and improvements section): https://python.langchain.com/docs/how_to/#retrievers

```Retriever``` interface wraps an index
- Uniform interface for interacting with different types of retrieval systems: Vector stores, Graph databases, Relational databases
- Input: Query string
- Output: List of LangChain Document objects
- Its a runnable (standard interface for LangChain components). Therefore, it has the method ```invoke()``` to invoke it with a query
- A vectorstore can be used as a retriever by calling the ```as_retriever()``` method
- https://python.langchain.com/docs/concepts/retrievers/

Query Analysis: Models transform or construct search queries from raw user input to optimise retrieval
- https://python.langchain.com/docs/concepts/retrieval/
- Query Re-writing
  - Multi-query
  - Decomposition
  - Step-back
  - HyDE
- Query Construction
  - Self Query (use metadata)

Information Retrieval: Search queries are used to fetch information from various retrieval systems.
- https://python.langchain.com/docs/concepts/retrieval/
- Lexical search indexes (based on keywords). Data structure to implement this is called inverted index. Lexical search algorithms include BM25
- Vector indexes (based on word embeddings)


Most common type of ```Retriever``` is the ```VectorStoreRetriever```: Uses the similarity search capabilities of a vector store to facilitate retrieval. Any VectorStore can easily be turned into a Retriever with ```VectorStore.as_retriever()```

<br/>
<br/>


<u>**Experimentation and Improvements**</u>
1. Ensemble/Hybrid Search: Combine multiple retrievers. Particularly useful when you have multiple retrievers that are good at finding different types of relevant documents. We could fist try combining a sparse retriever (BM25 that is keyword based) and dense retriever (embedding similarity)
  - https://python.langchain.com/docs/concepts/retrievers/
  - https://python.langchain.com/docs/how_to/ensemble_retriever/
  - https://python.langchain.com/docs/how_to/hybrid/

2. Source Document Retention Approaches: Retain a link between the transformed document (chunks) and original document, giving the retriever the ability to return the original document. For example, you may use small chunk size for indexing documents in a vectorstore. If you return only the chunks as the retrieval result, then the model will have lost the original document context for the chunks.
  - Multi-Vector retriever: https://python.langchain.com/docs/how_to/multi_vector/
  - ParentDocument retriever: https://python.langchain.com/docs/how_to/parent_document_retriever/

3. Query Rewriting Approaches:
  - https://python.langchain.com/docs/concepts/retrieval/
  - Multi-query
  - Decomposition
  - Step-back
  - HyDE

4. Query Construction Approaches:
  - https://python.langchain.com/docs/concepts/retrieval/
  - Self query

5. Others:
  - Contextual Compression
  - Reorder/Rank Documents
    - https://python.langchain.com/docs/how_to/long_context_reorder/
    - RSV, RSV with smoothing (assume relevance feedback)
    - IDF (assume relevance feedback)
    - BM25 (assume no relevance feedback)
  - Maximal marginal relevance?
  - Multi Vector Retrieval?: https://python.langchain.com/docs/how_to/multi_vector/

6. IR System Evaluation Metrics (Must do)
  - Binary Relevance
  - Multiple Levels of Relevance


A list of techniques in LangChain:
- https://python.langchain.com/docs/how_to/#retrievers


**Basic**

In [ ]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 10})

retrieved_docs = retriever.invoke("What are the best hikes in Norway?")

In [ ]:
retrieved_docs[0].page_content

"In many countries, the best hiking routes are only obtainable to those in the know or with high-tech equipment – this is not the case in Norway, the undisputed home of the fjords! However, with many of the best Norway fjord hikes accessible only by boat or kayak, we would recommend travelling with a guide where possible.\n\nHere are 15 fjords worth hiking on your next Norway walking adventure. These are all day hikes with breathtaking views, many of which are accessible by public transport.\n\n1. Preikestolen, Lysefjord\n\n2. Breiskrednosi Summit Hike, Naerøyfjord\n\n3. Romsdalseggen Ridge, Isfjorden\n\n4. Mount Skåla, Nordfjord\n\n5. Trolltunga (aka The Devil's Tongue), Hardangerfjord\n\n6. Aurlandsdalen Valley, Sognefjord\n\n7. Dronningstien, Hardangerfjord\n\n8. Vidasethovden, Sognefjord\n\n9. Mount Hanguren, Sognefjord/ Hardangerfjord\n\n10. Mount Fløyen, Sognefjord/ Hardangerfjord\n\n11. Mount Ulriken, Sognefjord/ Hardangerfjord\n\n12. Rimstigen, Næroyfjord\n\n13. Urkeegga, Hjøru

<br/>
<br/>
<br/>
<br/>
<br/>

## **Experimentation Guide**

At each step of the RAG pipeline, we take what work's best for our use case, and use those as the control variables for our next step for experimentation. This is to prevent an exhaustive, large amount of experiments.

other random thoughts
- does lower casing text help etc.
- experiment with embeddings in earlier parts or re-structure experimentations now that we've gone through a bit
- remember to use text splitter for the docs

helpful fn?

```
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )
```

<br/>
<br/>
<br/>
<br/>
<br/>

## **Part 5 Experimentation**

In the experimentation below, we mostly use "the FAISS vector store, with IndexFlatL2 as the hash and hamming distance as the similarity measure

Control variables:
- Vector store: FAISS Index Flat L2 (Except for self-query due to LangChain incompatibility)
  - Double confirm the similarity measure: https://python.langchain.com/api_reference/community/vectorstores/langchain_community.vectorstores.utils.DistanceStrategy.html#langchain_community.vectorstores.utils.DistanceStrategy
- Bi-encoder sentence embedding: ```sentence-transformers/all-mpnet-base-v2```
  - Maybe can do some experiments with this too: ```multi-qa-MiniLM-L6-cos-v1```

- Experiment with different indexes above also

- Move experiment data to a folder
- Remove reptitive set up code
- is it okay to use experiment data to compare performance

<u>**Setup Simple Experiment Data**</u>

### **Experiment 1: Hybrid/Ensemble**

- https://python.langchain.com/docs/how_to/ensemble_retriever/

Combine sparse retriever like BM25 that is based on keyword search with dense retriever that is based on embedding similarity/semantic similarity.

EnsembleRetrievers support ensembling of results from multiple retrievers (base type BaseRetriever).

They rerank the results of constituent retrievers based on the Reciprocal Rank Fusion algorithm (RRF).

This common approach, combining a basic keyword and similarity search, could be applicable to our use case, a Nordic-Region information retrieval system, to retrieve keyword-wise relevant (BM25) as well as semantically-similar (LSH with projections) results for a start.


In [ ]:
%pip install --quiet --upgrade bitsandbytes langchain langchain-community langchain-huggingface transformers beautifulsoup4 faiss-gpu rank_bm25 lark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.3 MB/s eta 0:00:00


In [ ]:
from langchain_core.documents import Document

# Simple experiment example data
docs = [
    Document(
        page_content="The best hikes in Norway include the Reinebringen hike in the Lofoten islands. At a modest 448 meters high, Reinebringen is far from one of the highest peaks on the Lofoten islands. Yet this is more than made up for by the iconic view from the summit of Reine. It is not suitable for winter! Also, the trail can be quite demanding as the steps are quite steep.",
        metadata={"activity": 'Hiking', "country": 'Norway'},
    ),
    Document(
        page_content="Unique hike that can be done are volcanic hikes which can be done in Iceland. It is recommended to go with a tour of experienced people!",
        metadata={"activity": 'Hiking', "country": 'Iceland'},
    ),
    Document(
        page_content="Popular food in Norway is seafood! The best seafood in the Nordic region can be found in Norway. The seafood is freshly caught from the arctic ocean. Popular choices include the famous norwegian salmon. Other delicacies include whale steak!",
        metadata={"activity": 'Food', "country": 'Norway'},
    ),
    Document(
        page_content="The famous street food of Iceland is the Hotdog! It is called the Baejarins Beztu Pylsur hot dog is made of a mix of lamb, beef and pork. Other delicacies of iceland include Fish and Chips as well as Tommi's burger.",
        metadata={"activity": 'Food', "country": 'Norway'},
    ),
    Document(
        page_content="Transportation within Reykjavik is fairly convenient as there is a public bus service called BSI. All you need to do is to download their mobile app, follow the instructions, and you're good to go. Transportation to places outside Reykjavik however requires a car. Some options include car rentals as well as booking bus tours.",
        metadata={"activity": 'Transportation', "country": 'Iceland'},
    ),
    Document(
        page_content="Finland is easily accessible with its HSL public transportation services where all you need to do is to download a mobile app and follow the instructions.",
        metadata={"activity": 'Transportation', "country": 'Iceland'},
    ),
    Document(
        page_content="Finland is known for its snowy-like landscape and captivating auroras. One of the best places to stay is the Glass huts in Skyfire village in Rovaniemi, Lapland where you can admire the beautiful northern lights and snowy landscape. The village has its very own restaurant called Sky Huts Restaurant and Bar which offers tailor-made menus by a professional chef using local ingredients.",
        metadata={"activity": 'Accomodation', "country": 'Finland'},
    ),
    Document(
        page_content="A nice place to stay in Norway is the Lofoten Islands, in particlar Unstad which provides a breathtaking view of the mountain valley, ocean, and if you're lucky, northern lights.",
        metadata={"activity": 'Accomodation', "country": 'Norway'},
    ),
]

In [ ]:
from langchain.retrievers import EnsembleRetriever # Supports Ensembling of results from multiple retrievers
from langchain_community.retrievers import BM25Retriever
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore

In [ ]:
# Initialise BM25 retreiver -> Ranked retrieval, Probabilistic IR Technique that is non-binary -> takes into account binary presence, but also frequency-related information
# https://python.langchain.com/docs/integrations/retrievers/bm25/
# https://www.kaggle.com/code/marcinrutecki/rag-ensemble-retriever-in-langchain/notebook#Key-Features-of-BM25
# https://pub.aimind.so/understanding-the-bm25-ranking-algorithm-19f6d45c6ce
# https://docs.vespa.ai/en/reference/bm25.html
# https://en.wikipedia.org/wiki/Okapi_BM25
# ranking function used in information retrieval systems to estimate the relevance of documents to a given search query.
bm25_retriever = BM25Retriever.from_documents(
    docs
)
bm25_retriever.k = 5 # num docs to return from BM_25

# Initialise the FAISS retriever -> Ranked retrieval, Similarity search using Locality Sensitive Hashing with Random Projections and Hamming Distance
# https://python.langchain.com/docs/integrations/vectorstores/faiss/
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

index = faiss.IndexFlatL2(len(embeddings_model.embed_query("hello world"))) # Initialise FAISS index with the dimensionality

faiss_vector_store = FAISS(
    embedding_function=embeddings_model,
    index=index, # what index to use
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

faiss_vector_store.add_documents(docs)
faiss_retriever = faiss_vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5}) # num docs to return from FAISS

# Initialise the hybrid/ensemble retriever
# Uses RRF to sum the rankings of each doc from both retrievers, discounting rankings that are lower.
# https://medium.com/@devalshah1619/mathematical-intuition-behind-reciprocal-rank-fusion-rrf-explained-in-2-mins-002df0cc5e2a
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

In [ ]:
docs = ensemble_retriever.invoke("What are the best hikes?")
docs

[Document(metadata={'activity': 'Hiking', 'country': 'Norway'}, page_content='The best hikes in Norway include the Reinebringen hike in the Lofoten islands. At a modest 448 meters high, Reinebringen is far from one of the highest peaks on the Lofoten islands. Yet this is more than made up for by the iconic view from the summit of Reine. It is not suitable for winter! Also, the trail can be quite demanding as the steps are quite steep.'),
 Document(metadata={'activity': 'Hiking', 'country': 'Iceland'}, page_content='Unique hike that can be done are volcanic hikes which can be done in Iceland. It is recommended to go with a tour of experienced people!'),
 Document(metadata={'activity': 'Accomodation', 'country': 'Finland'}, page_content='Finland is known for its snowy-like landscape and captivating auroras. One of the best places to stay is the Glass huts in Skyfire village in Rovaniemi, Lapland where you can admire the beautiful northern lights and snowy landscape. The village has its v

In [ ]:
docs = ensemble_retriever.invoke("What are the best food?")
docs

[Document(metadata={'activity': 'Hiking', 'country': 'Norway'}, page_content='The best hikes in Norway include the Reinebringen hike in the Lofoten islands. At a modest 448 meters high, Reinebringen is far from one of the highest peaks on the Lofoten islands. Yet this is more than made up for by the iconic view from the summit of Reine. It is not suitable for winter! Also, the trail can be quite demanding as the steps are quite steep.'),
 Document(metadata={'activity': 'Food', 'country': 'Norway'}, page_content='Popular food in Norway is seafood! The best seafood in the Nordic region can be found in Norway. The seafood is freshly caught from the arctic ocean. Popular choices include the famous norwegian salmon. Other delicacies include whale steak!'),
 Document(metadata={'activity': 'Accomodation', 'country': 'Finland'}, page_content='Finland is known for its snowy-like landscape and captivating auroras. One of the best places to stay is the Glass huts in Skyfire village in Rovaniemi, 

**Questions about Hybrid/Ensemble**
- For the food query, why is a hiking related document being returned first?
- How is the number of examples that are returned determined? (Not a priority)

<br/>
<br/>
<br/>

**Investigating BM25**

<u>Running BM25 with the full food query</u>

In [ ]:
bm25_retriever = BM25Retriever.from_documents(
    docs
)
bm25_retriever.k = 5

bm25_retriever.invoke("what are the best food?")

[Document(metadata={'activity': 'Hiking', 'country': 'Norway'}, page_content='The best hikes in Norway include the Reinebringen hike in the Lofoten islands. At a modest 448 meters high, Reinebringen is far from one of the highest peaks on the Lofoten islands. Yet this is more than made up for by the iconic view from the summit of Reine. It is not suitable for winter! Also, the trail can be quite demanding as the steps are quite steep.'),
 Document(metadata={'activity': 'Hiking', 'country': 'Iceland'}, page_content='Unique hike that can be done are volcanic hikes which can be done in Iceland. It is recommended to go with a tour of experienced people!'),
 Document(metadata={'activity': 'Food', 'country': 'Norway'}, page_content='Popular food in Norway is seafood! The best seafood in the Nordic region can be found in Norway. The seafood is freshly caught from the arctic ocean. Popular choices include the famous norwegian salmon. Other delicacies include whale steak!'),
 Document(metadata=

<u>Running BM25 with just the keyword that we want: food</u>

In [ ]:
bm25_retriever = BM25Retriever.from_documents(
    docs
)
bm25_retriever.k = 5

bm25_retriever.invoke("food")

[Document(metadata={'activity': 'Food', 'country': 'Norway'}, page_content='Popular food in Norway is seafood! The best seafood in the Nordic region can be found in Norway. The seafood is freshly caught from the arctic ocean. Popular choices include the famous norwegian salmon. Other delicacies include whale steak!'),
 Document(metadata={'activity': 'Food', 'country': 'Norway'}, page_content="The famous street food of Iceland is the Hotdog! It is called the Baejarins Beztu Pylsur hot dog is made of a mix of lamb, beef and pork. Other delicacies of iceland include Fish and Chips as well as Tommi's burger."),
 Document(metadata={'activity': 'Hiking', 'country': 'Iceland'}, page_content='Unique hike that can be done are volcanic hikes which can be done in Iceland. It is recommended to go with a tour of experienced people!'),
 Document(metadata={'activity': 'Accomodation', 'country': 'Norway'}, page_content="A nice place to stay in Norway is the Lofoten Islands, in particlar Unstad which p

**Conclusions about BM25**

- Why were the hiking documents ranked higher than the food documents with the full query?
  - Other terms in the query are also considered (stopwords are not removed) which will be factored into the simlarity BM25 calculation. For instance, the word 'best'
- Why are documents that do not contain the exact query term being returned?
  - No filtering of zero scores by LangChain's implementation


**How can we improve BM25**
- Drop stopwords from the query
  - https://www.linkedin.com/advice/0/what-role-stop-words-information-retrieval
- Consider: Do user queries more often contain words that are variations of keywords or do they more often contain the exact keyword? This will affect the weightage for our use case

**Improving BM25: Using NLTK stopwords**

In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
print(stopwords.words('english'))
nltk_stopwords = stopwords.words('english')

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import re
full_query = "What are the best food?"
# https://stackoverflow.com/questions/1751301/regex-match-entire-words-only
query_words = re.findall(r'\b\w+\b', full_query)

keywords = [keyword for keyword in query_words if keyword.lower() not in nltk_stopwords]

full_query_keywords_only = ' '.join(keywords)

print(f'The original query is {full_query}\n')
print(f'The new query is {full_query_keywords_only}\n')

bm25_retriever = BM25Retriever.from_documents(
    docs
)
bm25_retriever.k = 5

bm25_retriever.invoke(full_query_keywords_only)

The original query is What are the best food?

The new query is best food



[Document(metadata={'activity': 'Food', 'country': 'Norway'}, page_content='Popular food in Norway is seafood! The best seafood in the Nordic region can be found in Norway. The seafood is freshly caught from the arctic ocean. Popular choices include the famous norwegian salmon. Other delicacies include whale steak!'),
 Document(metadata={'activity': 'Food', 'country': 'Norway'}, page_content="The famous street food of Iceland is the Hotdog! It is called the Baejarins Beztu Pylsur hot dog is made of a mix of lamb, beef and pork. Other delicacies of iceland include Fish and Chips as well as Tommi's burger."),
 Document(metadata={'activity': 'Hiking', 'country': 'Iceland'}, page_content='Unique hike that can be done are volcanic hikes which can be done in Iceland. It is recommended to go with a tour of experienced people!'),
 Document(metadata={'activity': 'Accomodation', 'country': 'Norway'}, page_content="A nice place to stay in Norway is the Lofoten Islands, in particlar Unstad which p

We can see an improvement in the docs that we want to see using BM25 by removing stopwords

<br/>
<br/>
<br/>

**Investigating FAISS**

<u>Running FAISS retriever (semantic search: LSH with projections and hamming distance) with the full food query</u>

In [ ]:
faiss_retriever.invoke("What are the best food?")

[Document(metadata={'activity': 'Food', 'country': 'Norway'}, page_content='Popular food in Norway is seafood! The best seafood in the Nordic region can be found in Norway. The seafood is freshly caught from the arctic ocean. Popular choices include the famous norwegian salmon. Other delicacies include whale steak!'),
 Document(metadata={'activity': 'Food', 'country': 'Norway'}, page_content="The famous street food of Iceland is the Hotdog! It is called the Baejarins Beztu Pylsur hot dog is made of a mix of lamb, beef and pork. Other delicacies of iceland include Fish and Chips as well as Tommi's burger."),
 Document(metadata={'activity': 'Hiking', 'country': 'Norway'}, page_content='The best hikes in Norway include the Reinebringen hike in the Lofoten islands. At a modest 448 meters high, Reinebringen is far from one of the highest peaks on the Lofoten islands. Yet this is more than made up for by the iconic view from the summit of Reine. It is not suitable for winter! Also, the trail

**Thoughts about FAISS**

Seems like the embedding similarity works well (even though we use locality sensitive hashing with projections to increase the search speed by reducing vector resolution)

<br/>
<br/>
<br/>

**How can we improve the Ensemble/Hybrid Retrieval?**

- Remove stopwords. This also works for semantic search.
  - Possibly add stopwords unique to travel search queries well, thus expanding the stopword vocab

<br/>
<br/>
<br/>
<br/>
<br/>

### **Experiment 2: Query Construction (Self-Query)**

- https://python.langchain.com/docs/concepts/retrieval/
- https://python.langchain.com/docs/how_to/self_query/

Self-Query is a Query Construction technique which focuses on combining semantic search using natural language user queries together with specialised metadata filters.

It may be effective because users searching for information on holidays tend to search for information relating to a category, which we have included as metadata in our documents. Therefore, the questions that they ask may be better answered by fetching documents based on metadata rather than solely based on simiarity with text.
- This uses an LLM to transform user input into two things: (1) a string to look up semantically, (2) a metadata filter to go along with it. This is useful because oftentimes questions are about the METADATA of documents (not the content itself).

Rough steps of constructing Self-Query from scratch with LCEL
- Create a ```StructuredQuery``` object using an instruction template, given the doc desc, metadata info and user query, that is passed to an LLM for construction
- Translate the ```StructuredQuery``` object into a metadata filter in the syntax of the vector store we are using

In [ ]:
%pip install --quiet --upgrade bitsandbytes langchain langchain-community langchain-huggingface transformers beautifulsoup4 faiss-gpu rank_bm25 lark qdrant-client langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 31.2 MB/s eta 0:00:00

In [ ]:
from langchain_core.documents import Document

# Simple experiment example data
docs = [
    Document(
        page_content="The best hikes in Norway include the Reinebringen hike in the Lofoten islands. At a modest 448 meters high, Reinebringen is far from one of the highest peaks on the Lofoten islands. Yet this is more than made up for by the iconic view from the summit of Reine. It is not suitable for winter! Also, the trail can be quite demanding as the steps are quite steep.",
        metadata={"activity": 'Hiking', "country": 'Norway'},
    ),
    Document(
        page_content="Unique hike that can be done are volcanic hikes which can be done in Iceland. It is recommended to go with a tour of experienced people!",
        metadata={"activity": 'Hiking', "country": 'Iceland'},
    ),
    Document(
        page_content="Popular food in Norway is seafood! The best seafood in the Nordic region can be found in Norway. The seafood is freshly caught from the arctic ocean. Popular choices include the famous norwegian salmon. Other delicacies include whale steak!",
        metadata={"activity": 'Food', "country": 'Norway'},
    ),
    Document(
        page_content="The famous street food of Iceland is the Hotdog! It is called the Baejarins Beztu Pylsur hot dog is made of a mix of lamb, beef and pork. Other delicacies of iceland include Fish and Chips as well as Tommi's burger.",
        metadata={"activity": 'Food', "country": 'Norway'},
    ),
    Document(
        page_content="Transportation within Reykjavik is fairly convenient as there is a public bus service called BSI. All you need to do is to download their mobile app, follow the instructions, and you're good to go. Transportation to places outside Reykjavik however requires a car. Some options include car rentals as well as booking bus tours.",
        metadata={"activity": 'Transportation', "country": 'Iceland'},
    ),
    Document(
        page_content="Finland is easily accessible with its HSL public transportation services where all you need to do is to download a mobile app and follow the instructions.",
        metadata={"activity": 'Transportation', "country": 'Iceland'},
    ),
    Document(
        page_content="Finland is known for its snowy-like landscape and captivating auroras. One of the best places to stay is the Glass huts in Skyfire village in Rovaniemi, Lapland where you can admire the beautiful northern lights and snowy landscape. The village has its very own restaurant called Sky Huts Restaurant and Bar which offers tailor-made menus by a professional chef using local ingredients.",
        metadata={"activity": 'Accomodation', "country": 'Finland'},
    ),
    Document(
        page_content="A nice place to stay in Norway is the Lofoten Islands, in particlar Unstad which provides a breathtaking view of the mountain valley, ocean, and if you're lucky, northern lights.",
        metadata={"activity": 'Accomodation', "country": 'Norway'},
    ),
]

In [ ]:
from langchain.chains.query_constructor.base import AttributeInfo

# Provide info about the metadata fields that our doc support and a short desc of the doc contents
metadata_field_info = [
    AttributeInfo(
        name="activity",
        description="The activities mentioned in the article",
        type="string",
    ),
    AttributeInfo(
        name="country",
        description="The country that the article is talking about",
        type="string",
    )
]

document_content_description = "Summaries of what to do in a country"

In [ ]:
import torch
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import BitsAndBytesConfig

# Initialise the LLM
llm = HuggingFacePipeline(
      pipeline=pipeline(
        model="Qwen/Qwen2.5-3B-Instruct",
        task="text-generation",
        temperature=0.2,
        do_sample=True,
        repetition_penalty=1.1,
        max_new_tokens=400,
        device_map="auto"
      )
    )

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

# Initialise Chroma as a vector store for LangChain's self-query because FAISS is not supported
# https://python.langchain.com/docs/how_to/self_query/
# https://python.langchain.com/docs/integrations/retrievers/self_query/
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

chroma_vectorstore = Chroma.from_documents(docs, embeddings_model)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
chroma_retriever = SelfQueryRetriever.from_llm(
    llm, chroma_vectorstore, document_content_description, metadata_field_info, verbose=True
)

In [ ]:
chroma_retriever.invoke("What is the best food?")

OutputParserException: Parsing text
Your goal is to structure the user's query to match the request schema provided below.

<< Structured Request Schema >>
When responding use a markdown code snippet with a JSON object formatted in the following schema:

```json
{
    "query": string \ text string to compare to document contents
    "filter": string \ logical condition statement for filtering documents
}
```

The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.

A logical condition statement is composed of one or more comparison and logical operation statements.

A comparison statement takes the form: `comp(attr, val)`:
- `comp` (eq | ne | gt | gte | lt | lte): comparator
- `attr` (string):  name of attribute to apply the comparison to
- `val` (string): is the comparison value

A logical operation statement takes the form `op(statement1, statement2, ...)`:
- `op` (and | 

**Conclusions about using LLMs for Self-Query in LangChain**

Using LLMs to construct complex structures, especually the JSON StructuredQuery object required for Self-Query, is prone to errors
- Issue is listed here: https://github.com/langchain-ai/langchain/issues/5882

<br/>
<br/>
<br/>

**Can we create our own metadata filtering mechanism to leverage the usefulness of metadata?**

- Its too complex and we hit the same issue of needing to use LLMs: Need to leverage LLMs but as we have seen, LLMs with less parameters have trouble accomplishing this. Need to identify each word in the query and assign it for comparison in a filter. Even then, the word in the query may not be an exact match.

<br/>
<br/>
<br/>
<br/>
<br/>

### **Experiment 3: Query Re-writing/Query Expansion -> Decomposition**

- https://python.langchain.com/docs/concepts/retrieval/
- https://github.com/langchain-ai/rag-from-scratch/blob/main/rag_from_scratch_5_to_9.ipynb

Decomposition is a Query Re-writing technique that focuses decomposing a question into a set of subquestions.

This may be effective for our system as users planning a holiday tend may string together many requests in one question.

Paper
- https://arxiv.org/pdf/2212.10509
- https://arxiv.org/pdf/2205.10625

In [ ]:
%pip install --quiet --upgrade bitsandbytes langchain langchain-community langchain-huggingface transformers beautifulsoup4 faiss-gpu rank_bm25 lark qdrant-client langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 109.3 MB/s eta 0:00:

In [ ]:
from langchain_core.documents import Document

# Simple experiment example data
docs = [
    Document(
        page_content="The best hikes in Norway include the Reinebringen hike in the Lofoten islands. At a modest 448 meters high, Reinebringen is far from one of the highest peaks on the Lofoten islands. Yet this is more than made up for by the iconic view from the summit of Reine. It is not suitable for winter! Also, the trail can be quite demanding as the steps are quite steep.",
        metadata={"activity": 'Hiking', "country": 'Norway'},
    ),
    Document(
        page_content="Unique hike that can be done are volcanic hikes which can be done in Iceland. It is recommended to go with a tour of experienced people!",
        metadata={"activity": 'Hiking', "country": 'Iceland'},
    ),
    Document(
        page_content="Popular food in Norway is seafood! The best seafood in the Nordic region can be found in Norway. The seafood is freshly caught from the arctic ocean. Popular choices include the famous norwegian salmon. Other delicacies include whale steak!",
        metadata={"activity": 'Food', "country": 'Norway'},
    ),
    Document(
        page_content="The famous street food of Iceland is the Hotdog! It is called the Baejarins Beztu Pylsur hot dog is made of a mix of lamb, beef and pork. Other delicacies of iceland include Fish and Chips as well as Tommi's burger.",
        metadata={"activity": 'Food', "country": 'Norway'},
    ),
    Document(
        page_content="Transportation within Reykjavik is fairly convenient as there is a public bus service called BSI. All you need to do is to download their mobile app, follow the instructions, and you're good to go. Transportation to places outside Reykjavik however requires a car. Some options include car rentals as well as booking bus tours.",
        metadata={"activity": 'Transportation', "country": 'Iceland'},
    ),
    Document(
        page_content="Finland is easily accessible with its HSL public transportation services where all you need to do is to download a mobile app and follow the instructions.",
        metadata={"activity": 'Transportation', "country": 'Iceland'},
    ),
    Document(
        page_content="Finland is known for its snowy-like landscape and captivating auroras. One of the best places to stay is the Glass huts in Skyfire village in Rovaniemi, Lapland where you can admire the beautiful northern lights and snowy landscape. The village has its very own restaurant called Sky Huts Restaurant and Bar which offers tailor-made menus by a professional chef using local ingredients.",
        metadata={"activity": 'Accomodation', "country": 'Finland'},
    ),
    Document(
        page_content="A nice place to stay in Norway is the Lofoten Islands, in particlar Unstad which provides a breathtaking view of the mountain valley, ocean, and if you're lucky, northern lights.",
        metadata={"activity": 'Accomodation', "country": 'Norway'},
    ),
]

In [ ]:
from langchain.chains.query_constructor.base import AttributeInfo

# Provide info about the metadata fields that our doc support and a short desc of the doc contents
metadata_field_info = [
    AttributeInfo(
        name="activity",
        description="The activities mentioned in the article",
        type="string",
    ),
    AttributeInfo(
        name="country",
        description="The country that the article is talking about",
        type="string",
    )
]

document_content_description = "Summaries of what to do in a country"

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

# Prompt Decomposition template to help break a question into sub questions


template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries numbered 1 to 3, each on a new line, where each query ends with '?'):"""


# prompt_decomposition = ChatPromptTemplate.from_template(template) # more for chat like interactions
prompt_decomposition = PromptTemplate.from_template(template)

In [ ]:
# LLM
import torch
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import BitsAndBytesConfig

llm = HuggingFacePipeline(
      pipeline=pipeline(
        model="Qwen/Qwen2.5-3B-Instruct",
        task="text-generation",
        temperature=0.2,
        do_sample=True,
        repetition_penalty=1.1,
        max_new_tokens=400,
        device_map="auto"
      )
    )

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
from langchain_core.output_parsers import StrOutputParser
import re

def clean_questions(questions):
  questions = questions.strip()
  questions = questions.split('\n')
  questions_list = []
  for i in range(3):
    questions_list.append(questions[i].split('?')[0] + '?')
  return questions_list

# Chain
# add .bind(skip_prompt=True) to get response without prompt
# https://python.langchain.com/docs/integrations/llms/huggingface_pipelines/
generate_queries_decomposition = ( prompt_decomposition | llm.bind(skip_prompt=True) | StrOutputParser() | clean_questions)

# Run. Input the example question here
# Help me to plan a trip to Iceland
# How is the transportation and activities like in Norway
# When is the best time to go Finland and what is there to do
question = "When is the best time to go Finland and what is there to do"

# Apply the decompsition template
# Break down the questions into sub questions using the prompt decompsition pipeline
questions = generate_queries_decomposition.invoke({"question":question})

**Findings: LLMs trained on more parameters output better questions and follow the format better**
- Example vs Qwen2.5-0.5B-Instruct, Qwen2.5-1.5B-Instruct

In [ ]:
questions

['1) when is the best time to visit finland for tourism?',
 '2) what activities are available in finland during different seasons?',
 '3) how does weather affect tourist attractions in finland?']

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore

# Initialise the FAISS retriever -> Ranked retrieval, Similarity search using Locality Sensitive Hashing with Random Projections
# https://python.langchain.com/docs/integrations/vectorstores/faiss/
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

index = faiss.IndexFlatL2(len(embeddings_model.embed_query("hello world"))) # Initialise FAISS index with the dimensionality

faiss_vector_store = FAISS(
    embedding_function=embeddings_model,
    index=index, # what index to use
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

faiss_vector_store.add_documents(docs)
retriever = faiss_vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3}) # num docs to return from FAISS

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<u>**Answer Recursively**</u>

In [ ]:
# Prompt
# Template to recursively answer sub questions and build up the answers
# Might need to modify prompt to ask it to use only the context
'''
question: sub-question to be answered

q_a_pairs: Built up question-answer pairs that might be relevant

context: context retrieved for the current sub-question

Idea is to recursively answer each sub-question, using the current context and building upon previous answers to provide more comprehensive responses.
'''


template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question:

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

decomposition_prompt = PromptTemplate.from_template(template)

In [ ]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

# Utility function to format a given question and answer
def format_qa_pair(question, answer):
    """Format Q and A pair"""
    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()

# Initialise the q_a_pairs to be empty at first
q_a_pairs = ""

# For each sub-question that we decomposed from our main question earlier
for q in questions:
  rag_chain = (
  # Given {"question":q,"q_a_pairs":q_a_pairs}
  {"context": itemgetter("question") | retriever,  # Get the context relevant to the subquestion using the retriever
    "question": itemgetter("question"), # Get the subquestion
    "q_a_pairs": itemgetter("q_a_pairs")} # Get the built up qna pairs
  | decomposition_prompt # Pass the arguments into the template
  | llm.bind(skip_prompt=True)
  | StrOutputParser()) # Get the result from the LLM

  # Pass our rag chain the sub question and any prev built up q_a_pairs
  answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
  q_a_pair = format_qa_pair(q,answer) # Format it as sub_question, answer
  q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair # Update/Build the q_a_pairs

In [ ]:
answer

"To address the question of how weather affects tourist attractions in Finland, we will consider the provided context and background information.\n\nFirstly, let's examine the details about accommodations in Finland:\n\n1. **Snow Activities**: The Glass huts in Skyfire village in Rovaniemi, Lapland are highlighted as one of the best places to stay. These glass huts provide a unique experience where tourists can observe the Northern Lights and enjoy the snowy landscape. This indicates that the presence of snow and the occurrence of the Northern Lights significantly influence the appeal and accessibility of this attraction.\n\n2. **Northern Lights Viewing**: The Northern Lights, also known as Aurora Borealis, are mentioned as a captivating phenomenon that draws tourists to Finland. Snowfall conditions are crucial for visibility because they create a clear sky conducive to observing the aurora. Without sufficient snow cover, the atmosphere might be too cloudy or polluted, reducing the cha

<u>**Answer Individually**</u>

In [ ]:
# Answer each sub-question individually
from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

# RAG prompt
'''
https://smith.langchain.com/hub/rlm/rag-prompt

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Question: {question}

Context: {context}

Answer:
'''
prompt_rag = hub.pull("rlm/rag-prompt")
def retrieve_and_rag(question,prompt_rag,sub_question_generator_chain):
    """RAG on each sub-question"""

    # Use our decomposition /
    # Generate the sub questions using the chain
    sub_questions = sub_question_generator_chain.invoke({"question":question})

    # Initialize a list to hold RAG chain results
    rag_results = []

    for sub_question in sub_questions:

        # Retrieve documents for each sub-question
        retrieved_docs = retriever.get_relevant_documents(sub_question)

        # Use retrieved documents and sub-question in RAG chain
        # to answer the particular sub question
        answer = (prompt_rag | llm.bind(skip_prompt=True) | StrOutputParser()).invoke({"context": retrieved_docs,
                                                                "question": sub_question})

        # Append the answer to the sub question
        rag_results.append(answer)
    # Return the list of sub questions and their answers
    return rag_results,sub_questions

# Wrap the retrieval and RAG process in a RunnableLambda for integration into a chain
answers, questions = retrieve_and_rag(question, prompt_rag, generate_queries_decomposition)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
<ipython-input-38-9197ef3514dd>:32: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(sub_question)


In [ ]:
def format_qa_pairs(questions, answers):
    """Format Q and A pairs"""
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

# Format the list of sub questions and their answers from just now, to be formatted nicely by the LLM later
context = format_qa_pairs(questions, answers)

# Prompt
# Prompt template to use each individual sub-question and answer, as well as the main question
# to format a nice answer
template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt = PromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm.bind(skip_prompt=True)
    | StrOutputParser()
)

final_rag_chain.invoke({"context":context,"question":question})

"To answer your question, the best time to visit Finland for tourism is during winter months like December through February when the Northern Lights (auroras) are most visible due to long nights and cold temperatures. This period provides opportunities for activities like snowshoeing, skiing, and aurora watching. However, summer from June to August also offers unique experiences such as midnight sun and vibrant nature with activities including hiking, kayaking, and berry-picking. Additionally, there's a chance to experience the country's unique glass huts in Lapland during the winter season. Overall, both winter and summer offer distinct and memorable experiences in Finland. To get the most out of your trip, it's recommended to check local forecasts and plan accordingly based on your interests and preferences. Here’s a summary of the key points:\n\n- **Best Time to Visit:** Winter (December through February) for Northern Lights and snowy landscapes.\n- **Summer Activities:** Hiking, ka

**Issues that will be solved with Part 6**

There's some signs that the Model is hallucinating for the question: When is the best time to go Finland and what is there to do.

Solutions
- Add citations
- Prompt Engineering

<br/>
<br/>
<br/>

### **Experiment 4: Parent-Document retriever**

This may be effective for our Nordic Region Holiday Planner IR system because each document that is available for storage tends to be concentrated about an entire topic, to ensure as much helpful information as possible. For instance, general activities, accomodation, transportation etc.

Retrievers originally retrieve small relevant chunks. However, passing as much information to the retriever about these topics would be beneifical to the user, making Parent-Document retievers a good choice.
- Embed smaller chunks, retrieve larger chunks (parent document -> whole raw document or larger chunk)
- Need to balance between the amount of information because of context window requirements, or is this even a factor? Check inference speeds below

- https://python.langchain.com/docs/how_to/parent_document_retriever/

In [ ]:
%pip install --quiet --upgrade bitsandbytes langchain langchain-community langchain-huggingface transformers beautifulsoup4 faiss-gpu rank_bm25 lark qdrant-client langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 62.1 MB/s eta 0:00:0

In [ ]:
from langchain_core.documents import Document

# Simple experiment example data
docs = [
    Document(
        page_content='''
        10 new unique places to stay in Finland From modern wooden cabins and brand-new glass huts to resorts with a touch of luxury: Finland offers a plethora of unique accommodation options that showcase the country's breathtaking natural beauty both in the materials and the locations. In recent years, there has been upsurge in unique new hotels, resorts, and cabins opening up around the country. We've curated a list of the coolest and most recently opened ones where you can immerse yourself in Finnish nature and find your inner happiness. Article published in June 2023.; 1. Hilltop Forest – Inkoo, Helsinki region: Hilltop Forest, a small resort in Inkoo, offers charming triangular wooden huts with large windows facing a lush forest. Curl up in the fresh linen bedding and admire the view over the majestic pine trees and birches. The sole mission of the day is to stroll through the forest to the nearby spa. The resort's restaurant serves breakfast, lunch, freshly baked pizzas, wine, and forest-inspired dinners. Hilltop Forest is a perfect minivacation destination, just a under an hour-drive from Helsinki. Eat, sleep and repeat.; 2. The Torby – Fiskars village, Helsinki region: The brand new Torby Hotel, opened in June 2023, is nestled in the charming Fiskars Village. The Torby offers an enchanting retreat for travellers seeking tranquillity and inspiration. Set amidst the lush countryside of the Helsinki region, this boutique hotel seamlessly blends modern design with the rich cultural heritage of the Fiskars village. The village is a treasure trove of creativity, with its vibrant art studios, quaint shops, and idyllic gardens. Guests can explore the local craftsmanship, indulge in delicious Finnish cuisine, or simply immerse themselves in the serene ambiance. Whether seeking a peaceful retreat or a cultural adventure, the Torby Hotel and Fiskars Village promise an unforgettable experience.; 3. Hotel Vihannonkulma – Loimaa, Coast and Archipelago: The Hotel Vihannonkulma building, a private home of the car dealer Olli Vihanto, was closed for almost 30 years until the apartment was rediscovered and revived into a boutique hotel opened in 2022. Spend the night in a classic room with sleek Alvar Aalto inspired lines, or in a modern 70s-style room with eccentric details and teak furniture. Enjoy the small-town atmosphere of Loimaa and stroll around in cafés, shops, and historical buildings. Loimaa, located 50 minutes northeast of Turku, serves as a gateway to the surrounding forests and nature.; 4. Kanava Resort – Oravi, Lakeland: Located on a narrow headland surrounded by Lake Saimaa, Kanava Resort is a haven for happiness seekers and outdoor adventurers. The resort is a harmonious blend of contemporary design and natural beauty. The buildings, constructed with sustainable materials, seamlessly blend into the landscape, providing an immersive experience for guests. The resort offers a range of activities across the year. In the winter, take a ride on a snowmobile or slide down the terrain on a pair of skis. During summer hop on a canoe, grab the oars and slide through the waters. Make sure not to miss the most popular Lakeland adventure: spotting ringed seals together with a guide. Kanava Resort is just a short drive away from the national parks Linnansaari and Kolovesi. While you are in the area, remember to also visit the historical landmark Olavinlinna Castle and the iconic Punkaharju Ridge.; 5. PihlasResort – Joroinen, Lakeland: Located on the shore of the biggest lake in Finland, Lake Saimaa, PihlasResort is the perfect place to embrace luxury. The brand new suites, opening in summer 2023, are equipped with a kitchen, a private sauna, and a terrace facing a private garden. Relax by the lake, have a drink in a jacuzzi, or feel the rejuvenating power of an authentic Finnish sauna – relaxation is guaranteed. During your visit, dine in the resort's own restaurant Siimes, or enjoy a unique tea menu in the nearby TeaHouse of Wehmais.; 6. Villipeura’s Niliaitta – Kivijärvi, Lakeland: At Villipeura's holiday village in Kivijärvi, located less than two hours north of Jyväskylä in central Lakeland, unique birdhouse-like boxes are elevated on pillars in the midst of a charming Finnish forest. These uncommon holiday cottages draw inspiration from traditional food storage sheds that were built on wooden pillars to protect food from animals in the past. In the holiday village area, you have access to a range of activities, including tennis, frisbee golf, and fishing. Nature trails of the national parks Salamajärvi and Pyhä Häkki are just around the corner.; 7. Glass huts in Skyfire Village – Rovaniemi, Lapland: At Skyfire Village in Rovaniemi you may need to pinch yourself to figure out whether the view is real: the panoramic windows in the glass huts allow you to gaze at northern lights in autumn and winter, or the midnight sun in the summer directly from your bed. Skyfire Village is in a quiet location with no light pollution making it even easier to see the auroras. The village's own restaurant, Sky Hut Restaurant & Bar, offers tailor-made menus by a professional chef using local ingredients.; 8. Kurula’s – Pyhätunturi, Lapland: Kurula’s in Lapland's pristine Pyhä region is the perfect vacation getaway for those who want a combination of quiet relaxation and outdoors activities. In the summer, go on a hike in Finland’s oldest national park Pyhä-Luosto or ascend the fell with a skiing lift to admire the views. In the winter, embark on a husky or reindeer safari and hunt the northern lights. Your own private lakeside sauna in the suite is a perfect way to end a day full of adventures. Kurula is close to the popular outdoor and skiing resort of Pyhä.; 9. Arctra – Rovaniemi, Lapland: At Arctra's private luxury resort near Rovaniemi in Lapland, you can enjoy a glamorous vacation atop the mighty hill of Ollerovaara, with stunning views of arctic nature. Explore the landscape by foot, on skis, or on a husky sled. A private 20 hectare wilderness area will guarantee that there most likely won’t be another soul in sight. Cosy up in your stylish suite or at the spa area. Arctra is only a 15-minute drive from Rovaniemi airport. The cottage serves as an excellent home base to explore Rovaniemi as well as the surrounding wilderness.; 10. Cahkal Hotel – Kilpisjärvi, Lapland: Cahkal Hotel is located 400 kilometres above the Arctic Circle in the northwesternmost point of Finland, in the truly unique Nordic wilderness. Cahkal is an ideal destination for experiential travellers looking for a view – all the hotel rooms have exquisite mountain views. Embark on an expedition with a local guide to explore the arctic landscape and even pop by in Norway to see the fjords and the Arctic Ocean. Unwind in the hotel's sauna after an adventure of a lifetime. Cahkal's heating system operates by sustainable geothermal heating and solar panels provide visitors with green electricity.
        '''
    ),
    Document(
        page_content='''
        Finnish food culture Finland’s cuisine is built around fresh, natural ingredients gathered straight from the waters, fields and forests: Get to know the local ingredients and delicious dishes you simply can’t miss when visiting Finland. Finns are known as leaders of research and innovation. But when it comes to food, they tend to favour tradition. Finnish cuisine differs from neighbouring Scandinavian countries because it has one foot on either side of its borders – Russia in the East and Sweden in the West. Since day one, the Finnish diet has been built around surviving the harsh conditions of our northern climate. Growth seasons are dictated by the cold, meaning many local products are only available for a limited amount of time. These days, you can buy just about anything your heart desires in Finland. But go local while you’re here, and you’ll be in for a treat! Also, if you have the chance, continue your culinary journey to some of the best restaurants in the country.; Cereals and grains – made from scratch: All hail the golden crop of Finnish soil, oats! And what’s not to love about the latest boom of oat milks, creams and yoghurts, not to mention pulled oats, which are an extremely tasty alternative to meat? Make sure to taste this decidedly Finnish innovation! If you’re more into traditional foods, stick to good ol’ porridge. This hearty delight can be enjoyed at any gas station, hotel or café for just a few euros. Warming, filling and incredibly tasty, it’s best served with a spoonful of jam or a little fresh butter. Tip: When it comes to local grains, think bakery and check out pastry chef Teemu Aura. The bakeries carrying his name and signature pink bun tracks are called Pullabiili, and they can be found throughout the Helsinki region offering classic Finnish buns, flaky croissants and sourdough breads. All these treats are made from scratch and combine culinary creativity with high-quality ingredients.; Fish – another staple of the Finnish diet: Whether it’s tasty salmon soup, fillets of perch, pickled Baltic herring or smoked vendace, the list simply goes on and on. Best consumed fresh at food markets, delis and restaurants throughout the country, fish dishes are the heart of the Finnish diet. Unsure what to try? Go for smoked – it’s the archipelago’s signature way to enjoy the fruits of the sea. Or make some memories on a fishing trip, where you can catch and prepare your own. Contact the local tourist information points for tours. Tip: Crayfish parties, or “rapujuhlat” (“kräftskiva” in Swedish), are a Swedish tradition the Finns adopt during late summer’s crayfish season. These small fresh-water lobsters are considered a gourmet treat, which is why they are feted in style – often accompanied by plenty of schnapps and special crayfish songs. If you’re invited to one of these parties, do say yes!; Fresh, juicy berries are the gold of Finland’s forests: Plentiful and sweet when in season, lingonberries, woodland strawberries and blueberries taste best when they’re picked straight from the forest or purchased fresh by the litre at a local market. Off-season and outside of the warmer months of the year, local jams with cloudberries and the notorious sea-buckthorn are the preferred way to enjoy Finland’s berry bounty. Tip: Enjoy your berries dried and ground. METTÄ Nordic offers the exciting flavours of the Finnish forests in powder form. Buy a bag or two and try this delicious addition on breakfasts and snacks at home!; Mushrooms – ceps, chanterelles, false morel and list goes on..: Under the Everyman’s right, you can pick almost anything your heart desires while visiting Finland’s forests. Just be mindful and check the local rules if roaming in national parks or other protected areas. Tip: Please don’t pick mushrooms unless you’re an experienced forager. Guided mushroom-picking tours are always the safest bet for beginners.; Finnish superfood innovations have exploded onto the world’s markets: From Beanit’s plant-based protein products made from Nordic Fava beans to locally brewed kombucha drinks by Good Guys Kombucha in Pirkkala, Finland’s food companies are always thinking ahead. Tip: Want to try Finland’s superfoods? Go wild with an herb foraging class. You’ll not only return with a bag full of fresh greens, you’ll also get to try forest bathing, another Finnish. Foraging trips can be organised through companies such as Feel the Nature, and Finland, Naturally Experiences.   Hungry for more? Here’s a list of iconic foods and regional delicacies beloved by locals and visitors alike. Make sure to try them all!; Leipäjuusto – delicious dairy product: Leipäjuusto, known as “Squeaky Cheese,” is a mild, incredibly tasty cheese that’s typically made of cow’s milk. First, the milk is curdled, then it’s baked it in the oven, and finally, it’s cut into thin wedges and served. The exterior of the cheese gets its spotted black and white colouring from the heat of the oven, and it’s Finnish name means “cheese bread” (since it’s baked like bread). The end product is a yummy cheese with a deliciously squeaky consistency. Tip: Leipäjuusto is often served with cloudberry jam as dessert, but the traditional Sámi way of eating it is to dip the wedges in hot, black coffee. Try it to experience the perfect blend of smooth, fatty cheese cut by hot, bitter coffee.; Mild but full of flavour, Finnish salmon soup: Finnish salmon soup is a classic that’s served both at home and in restaurants. The most popular version features creamy white broth studded with salmon, onions and potatoes and garnished with a handful of dill. You’ll find this delicacy on the menu at many restaurants and cafés – it’s a true comfort food that will warm you up on a winter’s day.; Endless varieties of new potatoes: New potatoes with herring (silli). New potatoes with fresh lake fish and chantarelle sauce. New potatoes with fish roe (mäti). New potatoes with just a knob of butter, some dill and a little salt. The variations of new potatoes available in Finland are seemingly endless and certainly mouth-watering. You’ll find that Finns can talk about new potatoes forever, as the little spuds hold the promise of summer. Look for them starting around midsummer and expect to see statistics on the harvest in all the local papers.; Poronkäristys from Lapland: Reindeer are found in the northern province of Lapland, and according to recent research, their meat is one of the healthiest you can put on your plate. It’s high in B-12, omega-3 and omega-6 – and it’s lean and delicious! Served alongside mashed potatoes, sautéed reindeer is a Finnish treat that’s eaten throughout the country, all year round.; Kalakukko – a local delicacy: Describing this local delicacy is difficult, but it’s essentially a combination of salty vendace and fatty pork that’s then wrapped in a rye crust and baked in foil. Best served alone or with heaps of butter, this delicacy is most delicious at its origin: a busy marketplace in Kuopio in Finland’s Lakeland. Loosely translated as “fish rooster,” this iconic pie’s name makes absolutely no sense, but it tastes incredibly good! And fun fact: the name of this unique delicacy has been granted protection under the EU quality scheme, along with Karjalanpiirakka, or Karelian pie.; Karjalanpiirakka – a sublime pastry: And speaking of Karjalanpiirakka, Karelian pie is the crown jewel of Finnish cuisine. This sublime pastry is originally from the eastern province of Karelia, and with its filling of delicious rice, potatoes or carrots enclosed in an incredibly crisp rye crust, this buttery delicacy will win you over in no time. It’s best served with plenty of fresh butter or a serving of egg butter-spread. Tip: You’ll find Karelian pies at any self-respecting Finnish marketplace, café, supermarket or gas station. Fresh, unbaked pies can also be purchased and baked at home!; Ruisleipä – made from sour dough: Ruisleipä – or rye bread – is a staple of the Finnish diet. While there are many varieties, the most popular and widely available is reikäleipä, meaning “bread with a hole.” People used to hang their bread on poles from the rafters, and while it’s dense, flat and very heavy, many Finns will actually have it sent through the post while they’re living abroad. Näkkileipä is the cracker version of rye bread and there are many varieties, including the internationally-sold Finn Crisp cracker. These are eaten at breakfast with butter, cheese and other spreads, with soups at lunch or even as an evening snack.; Rieska – Finnish flatbreads: Rieska is made with dough from a variety of local grains that’s then shaped into a chapati-like bread (or “kovaohranen”). If they’re baked with potato dough, they’re called “lepuska.” The magic of this flatbread lies in its freshness. Prepared on the same day they’re consumed, these savoury delights go great with heaps of butter and scorching black coffee. Tip: Try a different flatbread in each place you visit. Hopefully, you’ll have the chance to taste the local Savo variety, which is made with sour milk.; Korvapuusti – a pastry enjoyed with a cup of coffee: Korvapuusti translates to “slapped ears” in English, but these pastries are essentially cinnamon buns. And while Finland doesn’t hold a patent on cinnamon buns, it probably should. Usually enjoyed with a cup of coffee (Finns consume more coffee and, perhaps, more cinnamon buns than any other European nation), it can be difficult to stop at just one. Or two.; Bilberry pie – adored by all: In July and August, bilberries paint the Finnish forest. You’ll want to pick and freeze them for winter (like the Finns do), but these berries are best enjoyed in the summer months, whether on their own or in homemade pies. Although most Finnish berries can be made into pie filling, bilberry pie, or mustikkapiirakka, served with fresh milk, is one that’s adored by all.; Chocolate from Finland: Chocolate isn’t exactly a Finnish invention, but it’s prepared and sold all over the country. One place to enjoy it is at Brunberg, a famous chocolate shop in Porvoo. In operation since 1871, this family-owned and operated business offers a wide variety of tasty treats. Favourites include Truffles and Kisses, which have become hits among those who love high-quality chocolate. Fazer, another family-operated chocolate brand, can be found in any Finnish café, supermarket or kiosk – their products are even sold as souvenirs in the airport’s duty-free shop. The company produces the country’s most iconic chocolate, Fazerin Sininen, which is loosely translated and commonly known as “the Blue.” This chocolate, along with Fazer’s licorices and other sweets, are frequently shipped abroad, and their company-operated cafés are scattered throughout Finland. They serve Karelian pies and salmon soup, too!
        '''
    ),
    Document(
        page_content='''
        Guide to driving in Finland Here are some tips for driving in Finland – rain, snow or shine: While Finland’s roads are typically in good condition, Nordic weather occasionally throws drivers a curveball. Whether you encounter reindeer on the road in Lapland or ice…well, pretty much anywhere, here’s some practical information for a smoother, safer road trip.; Renting a car: There are car hire companies in all major towns and cities, as well as at airports. It’s worth booking in advance, and you must have a credit card on you when you book. If you have an EU or Swiss drivers’ licence, you can hit the road in Finland straight away.; Avoiding traffic: Traffic jams aren’t common in Finland. Although you might hear some complaints about traffic from Finns, it’s rare to hear them from those who live in big cities elsewhere. To a Finn, a few-minute delay is the equivalent of heavy traffic.; Speed limits: Generally, the speed limit in Finland is 50 km/h in built-up areas and 80 km/h outside of them. Both limits are typically enforced if there is no other speed limit posted. On major highways in summer, you can drive 100 km/h. On motorways, the summertime limit is 120 km/h. In winter, however, the speed limit on highways is reduced to 80 km/h, and on motorways, it’s 100 km/h.; Turn your lights on!: If an oncoming vehicle flashes its high beams at you, it usually means you don’t have your headlights on. It’s a law in Finland that drivers must have their lights on – no matter the season and even in the midnight sun. But being flashed could also mean there’s an accident ahead or an animal is on the roadway, so be sure to stay alert.; Paying (or not paying) tolls: There are zero toll roads or bridges in Finland. Yes, zero. That’s because constructing and maintaining roads and highways is mostly funded by Finnish taxes. So go on, enjoy the drive – it’s on us!; Traffic enforcement cameras: Finnish police use automatic traffic surveillance equipment on heavily trafficked roads and in urban areas. You’ll know you’re driving on one of these roads if you see a yellow road sign depicting a camera.; Beware of the animals: Drivers in Finland should stay alert for signs warning of elk crossing the roadways. If you see one, slow down, and at night, use high beams whenever possible. While many of the country’s major highways are lined with high elk fences, accidents do happen – and they can be fatal for drivers, passengers and animals. Also, if you’re driving in Lapland, a reindeer (or twelve!) on the road is a very common sight. Please make sure you drive carefully and adjust your speed to account for limited visibility and potentially dangerous driving conditions.; Essentials of driving in winter: The downside of the delightful winter weather in Finland is that, at times, roads can be very icy. Because of this, all vehicles must be equipped with winter tyres between December 1st and March 1st. You might even want to have studded tires if you’re driving in the northern part of the country. Also, be sure to set aside adequate time for driving in winter. When the roads are icy or there’s a heavy snowfall, the only way to go about it safely is to drive slowly. Note that roads are generally not salted. Instead, they’re cleared by snow ploughs. When driving an electric car, be aware that cold weather and cabin heating cut your driving power. And whether your car is powered by electricity or gasoline, it’s always a good idea to wear or bring warm clothes with you when driving in Finland – just in case you need to make an unexpected stop.
        '''
    ),
]

In [ ]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore

**Linking the child chunks to the full parent document**
- https://python.langchain.com/docs/how_to/parent_document_retriever/

Note: During retrieval, ```ParentDocumentRetriever``` first fetches the small chunks but then looks up the parent ids for those chunks and returns those larger documents

In [ ]:
# This text splitter is used to create the child documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

# Initialise the FAISS retriever -> Ranked retrieval, Similarity search using Locality Sensitive Hashing with Random Projections and Hamming Distance
# https://python.langchain.com/docs/integrations/vectorstores/faiss/
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

index = faiss.IndexFlatL2(len(embeddings_model.embed_query("hello world"))) # Initialise FAISS index with the dimensionality

# The vectorstore to use to index the child chunks
vectorstore = FAISS(
    embedding_function=embeddings_model,
    index=index, # what index to use
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

# The storage layer for the parent documents
store = InMemoryStore()

# The retriever (empty at the start)
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter, # how to set how many is retrieved
    search_kwargs={"k": 2}
)

# Add documents
retriever.add_documents(docs, ids=None)

In [ ]:
# Just checking the behavour of fetching small chunks by similarity
sub_docs = vectorstore.similarity_search("best accomodation in finland")

In [ ]:
sub_docs

[Document(metadata={'doc_id': 'ee192f98-917f-49e1-963c-2ad77b498365'}, page_content="10 new unique places to stay in Finland From modern wooden cabins and brand-new glass huts to resorts with a touch of luxury: Finland offers a plethora of unique accommodation options that showcase the country's breathtaking natural beauty both in the materials and the locations. In recent years, there has been upsurge in unique new hotels, resorts, and cabins opening up around the"),
 Document(metadata={'doc_id': 'ee192f98-917f-49e1-963c-2ad77b498365'}, page_content="is the perfect place to embrace luxury. The brand new suites, opening in summer 2023, are equipped with a kitchen, a private sauna, and a terrace facing a private garden. Relax by the lake, have a drink in a jacuzzi, or feel the rejuvenating power of an authentic Finnish sauna – relaxation is guaranteed. During your visit, dine in the resort's own restaurant Siimes, or enjoy a unique tea menu in"),
 Document(metadata={'doc_id': 'ee192f98-

As we can see above, although relevant chunks are returned, not all information related to the best accomodation in finland is returned, our user could be missing out the accomodation of their choice!

In [ ]:
retrieved_docs = retriever.invoke("best accomodation in finland")

In [ ]:
retrieved_docs

[Document(metadata={}, page_content="\n        10 new unique places to stay in Finland From modern wooden cabins and brand-new glass huts to resorts with a touch of luxury: Finland offers a plethora of unique accommodation options that showcase the country's breathtaking natural beauty both in the materials and the locations. In recent years, there has been upsurge in unique new hotels, resorts, and cabins opening up around the country. We've curated a list of the coolest and most recently opened ones where you can immerse yourself in Finnish nature and find your inner happiness. Article published in June 2023.; 1. Hilltop Forest – Inkoo, Helsinki region: Hilltop Forest, a small resort in Inkoo, offers charming triangular wooden huts with large windows facing a lush forest. Curl up in the fresh linen bedding and admire the view over the majestic pine trees and birches. The sole mission of the day is to stroll through the forest to the nearby spa. The resort's restaurant serves breakfas

In [ ]:
len(retrieved_docs)

1

**Linking the child chunks to the a larger chunk of the parent document**

Due to the context-size and speed of the LLM, it may be better to trade-off some recall for faster speed

Here, we split the raw document into larger chunks as the parent document

We also split the raw document into smaller chunks and index it for similarity search

On retrieval we retrieve the larger chunks (but still not the full documents)

In [ ]:
# This text splitter is used to create the parent documents
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000) # Set the chunk size to define how large our 'Parent' document is

# This text splitter is used to create the child documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

# Initialise the FAISS retriever -> Ranked retrieval, Similarity search using Locality Sensitive Hashing with Random Projections and Hamming Distance
# https://python.langchain.com/docs/integrations/vectorstores/faiss/
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

index = faiss.IndexFlatL2(len(embeddings_model.embed_query("hello world"))) # Initialise FAISS index with the dimensionality

# The vectorstore to use to index the child chunks
vectorstore = FAISS(
    embedding_function=embeddings_model,
    index=index, # what index to use
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

# The storage layer for the parent documents
store = InMemoryStore()

# The retriever (empty at the start)
retriever_2 = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter, # how to set how many is retrieved
    parent_splitter=parent_splitter,
    search_kwargs={"k": 1}
)

# Add documents
retriever_2.add_documents(docs, ids=None)

In [ ]:
# Just checking the behavour of fetching small chunks by similarity
sub_docs_2 = vectorstore.similarity_search("best accomodation in finland")

In [ ]:
sub_docs_2

[Document(metadata={'doc_id': 'f23e621c-bcbf-48f8-86ba-aedb91b219b0'}, page_content="10 new unique places to stay in Finland From modern wooden cabins and brand-new glass huts to resorts with a touch of luxury: Finland offers a plethora of unique accommodation options that showcase the country's breathtaking natural beauty both in the materials and the locations. In recent years, there has been upsurge in unique new hotels, resorts, and cabins opening up around the country. We've"),
 Document(metadata={'doc_id': '9a864bbe-f226-4823-94b0-2b8c5bfefe85'}, page_content='visit the historical landmark Olavinlinna Castle and the iconic Punkaharju Ridge.; 5. PihlasResort – Joroinen, Lakeland: Located on the shore of the biggest lake in Finland, Lake Saimaa, PihlasResort is the perfect place to embrace luxury. The brand new suites, opening in summer 2023, are equipped with a kitchen, a private sauna, and a terrace facing a private garden. Relax by the lake, have a'),
 Document(metadata={'doc_id

In [ ]:
retrieved_docs_2 = retriever_2.invoke("best accomodation in finland")

In [ ]:
retrieved_docs_2

[Document(metadata={}, page_content="10 new unique places to stay in Finland From modern wooden cabins and brand-new glass huts to resorts with a touch of luxury: Finland offers a plethora of unique accommodation options that showcase the country's breathtaking natural beauty both in the materials and the locations. In recent years, there has been upsurge in unique new hotels, resorts, and cabins opening up around the country. We've curated a list of the coolest and most recently opened ones where you can immerse yourself in Finnish nature and find your inner happiness. Article published in June 2023.; 1. Hilltop Forest – Inkoo, Helsinki region: Hilltop Forest, a small resort in Inkoo, offers charming triangular wooden huts with large windows facing a lush forest. Curl up in the fresh linen bedding and admire the view over the majestic pine trees and birches. The sole mission of the day is to stroll through the forest to the nearby spa. The resort's restaurant serves breakfast, lunch, 

We can see above that we now return 'Parent' Documents

**Comparing inference speeds with full parent doc vs chunk of a parent doc**

In [ ]:
# LLM
import torch
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import BitsAndBytesConfig

llm = HuggingFacePipeline(
      pipeline=pipeline(
        model="Qwen/Qwen2.5-3B-Instruct",
        task="text-generation",
        temperature=0.2,
        do_sample=True,
        repetition_penalty=1.1,
        max_new_tokens=400,
        device_map="auto"
      )
    )

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
from langchain import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Question: {question}
Context: {context}
Helpful Answer:
"""

prompt = PromptTemplate.from_template(template)

In [ ]:
from langchain_core.runnables import RunnableLambda
output_parser = StrOutputParser()

def print_context_length(inputs):
    context_length = len(inputs['context'][0].page_content)
    print(f"Context length: {context_length}")
    return inputs

chain_full_parent = (
    {"context": (lambda x: x["question"]) | retriever,

     "question": (lambda x: x["question"])}
    | RunnableLambda(print_context_length)
    | prompt
    | llm.bind(skip_prompt=True)
    | StrOutputParser()
)

chain_chunk_parent = (
    {"context": (lambda x: x["question"]) | retriever_2, # using just 1 doc chunk (edit search_kwargs)
     "question": (lambda x: x["question"])}
    | RunnableLambda(print_context_length)
    | prompt
    | llm.bind(skip_prompt=True)
    | StrOutputParser()
)

In [ ]:
import time

In [ ]:
start = time.time()
answer = chain_full_parent.invoke({"question":"Please provide me with the best accomodation in Finland"})
end = time.time()
full_parent_inference_time = end-start
print(f'The inference time using the context as the full parent document is {full_parent_inference_time}')

Context length: 6984
The inference time using the context as the full parent document is 182.35966539382935


In [ ]:
answer

"Based on the information provided, the best accommodations in Finland could vary depending on what type of experience you're looking for. However, if you're interested in luxurious and scenic stays, some top choices include:\n\n1. **Kanava Resort** - Located on a narrow headland surrounded by Lake Saimaa, offering a harmonious blend of contemporary design and natural beauty. It features a range of activities throughout the year, including winter sports like snowmobiling and skiing, and summer activities such as canoeing.\n\n2. **PihlasResort** - Situated on the shore of Lake Saimaalä, known for its luxury suites with kitchens, private saunas, and terraces overlooking a private garden. It provides a relaxing environment with various dining options.\n\n3. **Arctra** - Nestled near Rovaniemi in Lapland, featuring a private luxury resort with stunning views of arctic nature. Activities include hiking, skiing, and sledding, along with a private wilderness area.\n\nThese three options offer

In [ ]:
start = time.time()
answer = chain_chunk_parent.invoke({"question":"Please provide me with the best accomodation in Finland"})
end = time.time()
chunk_parent_inference_time = end-start
print(f'The inference time using the context as the chunk parent document is {chunk_parent_inference_time}')

Context length: 1983
The inference time using the context as the chunk parent document is 174.22468638420105


In [ ]:
answer

'Based on the information provided in the context, it seems like the best accommodations in Finland could vary depending on personal preferences such as location, style, and amenities. However, two notable examples mentioned are:\n\n1. **Hilltop Forest - Inkoo**: This is described as a charming resort offering triangular wooden huts with large windows, set amidst a lush forest near Helsinki. It includes a nearby spa and features a variety of dining options.\n\n2. **The Torby - Fiskars village**: A newly opened boutique hotel located in the picturesque Fiskars village, known for its creative atmosphere, including art studios, shops, and gardens. It provides a tranquil setting for travelers looking for inspiration and cultural experiences.\n\nFor the best recommendation, I would need more specific details about what type of accommodation suits your needs (e.g., budget, preferred activities, proximity to attractions) and interests. Based solely on the given information, these two seem par

Need to run an average, not much difference to be honest

But need to check on context window of LLM

Is it necessary to checkout "Associating summaries with a document for retrieval"? Because we are also passing the document to an LLM for reasoning
- https://python.langchain.com/docs/how_to/multi_vector/#associating-summaries-with-a-document-for-retrieval
- Maybe this is helpful because of context stuffing

<br/>
<br/>
<br/>

### **Experiment 5: Re-ranking Strategies**

To ensure that our LLM recieves the most relevant retrieved documents from the retriever, we can re-rank the results from the retriever

**Retrieval Bi-Encoder + Re-Ranker Cross-Encoder**

Bi-Encoders produce a sentence embedding for a single sentence. This is what we have been doing with ```sentence-transformers```, where we pass it independently the document and query. These individual embeddings u and v are then compared using a similarity measure.

In Cross-Encoders, we pass both sentences simultaneously to the Transformer network. It then produces an output value between 0 and 1 indicating the similarity of the input sentence pair.

<u>When to use what</u>

Bi-encoders should be used when we need a sentence embedding in a vector space for efficient comparison, example information retrieval. Cross-encoders are the wrong choice for this application as the number of pairwise comparisons are very large.
- Lexical search (keywords): Look for literal matches of the query words. Does not recognize synonyms, acronyms or spelling variations
- Semantic search (dense retrieval): Encodes search query into vector space and retrieves doc embeddings that are close in vector space
- May still return irrelevant candidates

Although Cross-encoders achieve a higher performance than Bi-encoders, as mentioned, they do not scale well for large datasets as they are very slow. For information retrieval scenarios, we should first use an efficient Bi-Encoder to embed the documents for similarity measurement. Cross-Encoder should then be used to re-rank the top k results by computing the score for every (query,hit) combination.
- Re-ranker based on Cross-Encoder that scores relevacy of all candidates for a given search query
- Output is a ranked list of hits that we can present to the user
- Pass the query and possible document simultaneously to the transformer network which then outputs a single score between 0 and 1 indicating how relevant the document is for the given query
- Benefit: Higher performance as it performs attention across the query and document, but at the cost of some speed

Cross Encoder models by sbert
- https://www.sbert.net/docs/pretrained-models/ce-msmarco.html

- https://www.sbert.net/examples/applications/cross-encoder/README.html
- https://www.sbert.net/examples/applications/retrieve_rerank/README.html
- https://medium.com/@mpuig/bi-encoders-and-cross-encoders-two-sides-of-the-retrieval-coin-06a95fe18619
- https://python.langchain.com/docs/integrations/document_transformers/cross_encoder_reranker/
- https://towardsdatascience.com/how-to-use-re-ranking-for-better-llm-rag-retrieval-243f89414266

<u>Elaboration:</u>

2-step retrieval using bi-encoder model for initial candidate retrieval and cross-encoder model for re-ranking

First retrieve candidate documents using bi-encoder and embedding similarity search quickly (but less accurate). Then we re-rank these documents using a slower, but more accurate approach.

Since we could have thousands of documents in our database — and each document consists of many chunks of text — this process needs to be efficient and fast.
- Embedding vectors are a compressed representation of text, which ultimately leads to information loss.
- Using embeddings for similarity search is fast, but not the most reliable retrieval technique.

Context-stuffing problem: Documents can get “lost in the middle” of our context if we just stuff them in there. So, the best document matches should be at the very beginning of the context
- This is where re-ranking comes in

A cross-encoder approach processes 2 input text pairs simultaneously: Estimate a score of how relevant each candidate text is to a given text query

<u>Additional Research:</u>

To ensure fast search times at scale, we typically use vector search — that is, we transform our text into vectors, place them all into a vector space, and compare their proximity to a query vector using a similarity metric like cosine similarity.

For vector search to work, we need vectors. These vectors are essentially compressions of the "meaning" behind some text into (typically) 768 or 1536-dimensional vectors. There is some information loss because we're compressing this information into a single vector.

Because of this information loss, we often see that the top three (for example) vector search documents will miss relevant information. Unfortunately, the retrieval may return relevant information below our top_k cutoff. What do we do if relevant information at a lower position would help our LLM formulate a better response? The easiest approach is to increase the number of documents we're returning (increase top_k) and pass them all to the LLM.

LLMs have limits on how much text we can pass to them — we call this limit the context window. Some LLMs have huge context windows, like Anthropic's Claude, with a context window of 100K tokens. With that, we could fit many tens of pages of text — so could we return many documents (not quite all) and "stuff" the context window to improve recall? Again, no. We cannot use context stuffing because this reduces the LLM's recall performance — note that this is the LLM recall, which is different from the retrieval recall we have been discussing so far
- When storing information in the middle of a context window, an LLM's ability to recall that information becomes worse than had it not been provided in the first place

LLM recall refers to the ability of an LLM to find information from the text placed within its context window. Research shows that LLM recall degrades as we put more tokens in the context window. LLMs are also less likely to follow instructions as we stuff the context window — so context stuffing is a bad idea

<u>Solution</u>:

We can increase the number of documents returned by our vector DB to increase retrieval recall, but we cannot pass these to our LLM without damaging LLM recall.

The solution to this issue is to maximize retrieval recall by retrieving plenty of documents and then maximize LLM recall by minimizing the number of documents that make it to the LLM. To do that, we reorder retrieved documents and keep just the most relevant for our LLM — to do that, we use reranking.

Reranking models (aka cross encoders? So are cross-encoders the same as the LLMs below?): Model that, given a query and document pair, will output a similarity score. We use this score to reorder the documents by relevance to our query.
- We use two stages because retrieving a small set of documents from a large dataset is much faster than reranking a large set of documents — we'll discuss why this is the case soon — but TL;DR, rerankers are slow, and retrievers are fast.
- If a reranker is so much slower, why bother using them? The answer is that rerankers are much more accurate than embedding models.
- The intuition behind a bi-encoder's inferior accuracy is that bi-encoders must compress all of the possible meanings of a document into a single vector — meaning we lose information. Additionally, bi-encoders have no context on the query because we don't know the query until we receive it (we create embeddings before user query time).
- On the other hand, a reranker can receive the raw information directly into the large transformer computation, meaning less information loss. Because we are running the reranker at user query time, we have the added benefit of analyzing our document's meaning specific to the user query — rather than trying to produce a generic, averaged meaning.
- Rerankers avoid the information loss of bi-encoders — but they come with a different penalty — time.
- https://www.pinecone.io/learn/series/rag/rerankers/



In [ ]:
%pip install --quiet --upgrade bitsandbytes langchain langchain-community langchain-huggingface transformers beautifulsoup4 faiss-gpu rank_bm25 lark qdrant-client langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 87.8 MB/s eta 0:00:0

In [ ]:
from langchain_core.documents import Document

# Simple experiment example data
docs = [
    Document(
        page_content='''
        10 new unique places to stay in Finland From modern wooden cabins and brand-new glass huts to resorts with a touch of luxury: Finland offers a plethora of unique accommodation options that showcase the country's breathtaking natural beauty both in the materials and the locations. In recent years, there has been upsurge in unique new hotels, resorts, and cabins opening up around the country. We've curated a list of the coolest and most recently opened ones where you can immerse yourself in Finnish nature and find your inner happiness. Article published in June 2023.; 1. Hilltop Forest – Inkoo, Helsinki region: Hilltop Forest, a small resort in Inkoo, offers charming triangular wooden huts with large windows facing a lush forest. Curl up in the fresh linen bedding and admire the view over the majestic pine trees and birches. The sole mission of the day is to stroll through the forest to the nearby spa. The resort's restaurant serves breakfast, lunch, freshly baked pizzas, wine, and forest-inspired dinners. Hilltop Forest is a perfect minivacation destination, just a under an hour-drive from Helsinki. Eat, sleep and repeat.; 2. The Torby – Fiskars village, Helsinki region: The brand new Torby Hotel, opened in June 2023, is nestled in the charming Fiskars Village. The Torby offers an enchanting retreat for travellers seeking tranquillity and inspiration. Set amidst the lush countryside of the Helsinki region, this boutique hotel seamlessly blends modern design with the rich cultural heritage of the Fiskars village. The village is a treasure trove of creativity, with its vibrant art studios, quaint shops, and idyllic gardens. Guests can explore the local craftsmanship, indulge in delicious Finnish cuisine, or simply immerse themselves in the serene ambiance. Whether seeking a peaceful retreat or a cultural adventure, the Torby Hotel and Fiskars Village promise an unforgettable experience.; 3. Hotel Vihannonkulma – Loimaa, Coast and Archipelago: The Hotel Vihannonkulma building, a private home of the car dealer Olli Vihanto, was closed for almost 30 years until the apartment was rediscovered and revived into a boutique hotel opened in 2022. Spend the night in a classic room with sleek Alvar Aalto inspired lines, or in a modern 70s-style room with eccentric details and teak furniture. Enjoy the small-town atmosphere of Loimaa and stroll around in cafés, shops, and historical buildings. Loimaa, located 50 minutes northeast of Turku, serves as a gateway to the surrounding forests and nature.; 4. Kanava Resort – Oravi, Lakeland: Located on a narrow headland surrounded by Lake Saimaa, Kanava Resort is a haven for happiness seekers and outdoor adventurers. The resort is a harmonious blend of contemporary design and natural beauty. The buildings, constructed with sustainable materials, seamlessly blend into the landscape, providing an immersive experience for guests. The resort offers a range of activities across the year. In the winter, take a ride on a snowmobile or slide down the terrain on a pair of skis. During summer hop on a canoe, grab the oars and slide through the waters. Make sure not to miss the most popular Lakeland adventure: spotting ringed seals together with a guide. Kanava Resort is just a short drive away from the national parks Linnansaari and Kolovesi. While you are in the area, remember to also visit the historical landmark Olavinlinna Castle and the iconic Punkaharju Ridge.; 5. PihlasResort – Joroinen, Lakeland: Located on the shore of the biggest lake in Finland, Lake Saimaa, PihlasResort is the perfect place to embrace luxury. The brand new suites, opening in summer 2023, are equipped with a kitchen, a private sauna, and a terrace facing a private garden. Relax by the lake, have a drink in a jacuzzi, or feel the rejuvenating power of an authentic Finnish sauna – relaxation is guaranteed. During your visit, dine in the resort's own restaurant Siimes, or enjoy a unique tea menu in the nearby TeaHouse of Wehmais.; 6. Villipeura’s Niliaitta – Kivijärvi, Lakeland: At Villipeura's holiday village in Kivijärvi, located less than two hours north of Jyväskylä in central Lakeland, unique birdhouse-like boxes are elevated on pillars in the midst of a charming Finnish forest. These uncommon holiday cottages draw inspiration from traditional food storage sheds that were built on wooden pillars to protect food from animals in the past. In the holiday village area, you have access to a range of activities, including tennis, frisbee golf, and fishing. Nature trails of the national parks Salamajärvi and Pyhä Häkki are just around the corner.; 7. Glass huts in Skyfire Village – Rovaniemi, Lapland: At Skyfire Village in Rovaniemi you may need to pinch yourself to figure out whether the view is real: the panoramic windows in the glass huts allow you to gaze at northern lights in autumn and winter, or the midnight sun in the summer directly from your bed. Skyfire Village is in a quiet location with no light pollution making it even easier to see the auroras. The village's own restaurant, Sky Hut Restaurant & Bar, offers tailor-made menus by a professional chef using local ingredients.; 8. Kurula’s – Pyhätunturi, Lapland: Kurula’s in Lapland's pristine Pyhä region is the perfect vacation getaway for those who want a combination of quiet relaxation and outdoors activities. In the summer, go on a hike in Finland’s oldest national park Pyhä-Luosto or ascend the fell with a skiing lift to admire the views. In the winter, embark on a husky or reindeer safari and hunt the northern lights. Your own private lakeside sauna in the suite is a perfect way to end a day full of adventures. Kurula is close to the popular outdoor and skiing resort of Pyhä.; 9. Arctra – Rovaniemi, Lapland: At Arctra's private luxury resort near Rovaniemi in Lapland, you can enjoy a glamorous vacation atop the mighty hill of Ollerovaara, with stunning views of arctic nature. Explore the landscape by foot, on skis, or on a husky sled. A private 20 hectare wilderness area will guarantee that there most likely won’t be another soul in sight. Cosy up in your stylish suite or at the spa area. Arctra is only a 15-minute drive from Rovaniemi airport. The cottage serves as an excellent home base to explore Rovaniemi as well as the surrounding wilderness.; 10. Cahkal Hotel – Kilpisjärvi, Lapland: Cahkal Hotel is located 400 kilometres above the Arctic Circle in the northwesternmost point of Finland, in the truly unique Nordic wilderness. Cahkal is an ideal destination for experiential travellers looking for a view – all the hotel rooms have exquisite mountain views. Embark on an expedition with a local guide to explore the arctic landscape and even pop by in Norway to see the fjords and the Arctic Ocean. Unwind in the hotel's sauna after an adventure of a lifetime. Cahkal's heating system operates by sustainable geothermal heating and solar panels provide visitors with green electricity.
        '''
    ),
    Document(
        page_content='''
        Finnish food culture Finland’s cuisine is built around fresh, natural ingredients gathered straight from the waters, fields and forests: Get to know the local ingredients and delicious dishes you simply can’t miss when visiting Finland. Finns are known as leaders of research and innovation. But when it comes to food, they tend to favour tradition. Finnish cuisine differs from neighbouring Scandinavian countries because it has one foot on either side of its borders – Russia in the East and Sweden in the West. Since day one, the Finnish diet has been built around surviving the harsh conditions of our northern climate. Growth seasons are dictated by the cold, meaning many local products are only available for a limited amount of time. These days, you can buy just about anything your heart desires in Finland. But go local while you’re here, and you’ll be in for a treat! Also, if you have the chance, continue your culinary journey to some of the best restaurants in the country.; Cereals and grains – made from scratch: All hail the golden crop of Finnish soil, oats! And what’s not to love about the latest boom of oat milks, creams and yoghurts, not to mention pulled oats, which are an extremely tasty alternative to meat? Make sure to taste this decidedly Finnish innovation! If you’re more into traditional foods, stick to good ol’ porridge. This hearty delight can be enjoyed at any gas station, hotel or café for just a few euros. Warming, filling and incredibly tasty, it’s best served with a spoonful of jam or a little fresh butter. Tip: When it comes to local grains, think bakery and check out pastry chef Teemu Aura. The bakeries carrying his name and signature pink bun tracks are called Pullabiili, and they can be found throughout the Helsinki region offering classic Finnish buns, flaky croissants and sourdough breads. All these treats are made from scratch and combine culinary creativity with high-quality ingredients.; Fish – another staple of the Finnish diet: Whether it’s tasty salmon soup, fillets of perch, pickled Baltic herring or smoked vendace, the list simply goes on and on. Best consumed fresh at food markets, delis and restaurants throughout the country, fish dishes are the heart of the Finnish diet. Unsure what to try? Go for smoked – it’s the archipelago’s signature way to enjoy the fruits of the sea. Or make some memories on a fishing trip, where you can catch and prepare your own. Contact the local tourist information points for tours. Tip: Crayfish parties, or “rapujuhlat” (“kräftskiva” in Swedish), are a Swedish tradition the Finns adopt during late summer’s crayfish season. These small fresh-water lobsters are considered a gourmet treat, which is why they are feted in style – often accompanied by plenty of schnapps and special crayfish songs. If you’re invited to one of these parties, do say yes!; Fresh, juicy berries are the gold of Finland’s forests: Plentiful and sweet when in season, lingonberries, woodland strawberries and blueberries taste best when they’re picked straight from the forest or purchased fresh by the litre at a local market. Off-season and outside of the warmer months of the year, local jams with cloudberries and the notorious sea-buckthorn are the preferred way to enjoy Finland’s berry bounty. Tip: Enjoy your berries dried and ground. METTÄ Nordic offers the exciting flavours of the Finnish forests in powder form. Buy a bag or two and try this delicious addition on breakfasts and snacks at home!; Mushrooms – ceps, chanterelles, false morel and list goes on..: Under the Everyman’s right, you can pick almost anything your heart desires while visiting Finland’s forests. Just be mindful and check the local rules if roaming in national parks or other protected areas. Tip: Please don’t pick mushrooms unless you’re an experienced forager. Guided mushroom-picking tours are always the safest bet for beginners.; Finnish superfood innovations have exploded onto the world’s markets: From Beanit’s plant-based protein products made from Nordic Fava beans to locally brewed kombucha drinks by Good Guys Kombucha in Pirkkala, Finland’s food companies are always thinking ahead. Tip: Want to try Finland’s superfoods? Go wild with an herb foraging class. You’ll not only return with a bag full of fresh greens, you’ll also get to try forest bathing, another Finnish. Foraging trips can be organised through companies such as Feel the Nature, and Finland, Naturally Experiences.   Hungry for more? Here’s a list of iconic foods and regional delicacies beloved by locals and visitors alike. Make sure to try them all!; Leipäjuusto – delicious dairy product: Leipäjuusto, known as “Squeaky Cheese,” is a mild, incredibly tasty cheese that’s typically made of cow’s milk. First, the milk is curdled, then it’s baked it in the oven, and finally, it’s cut into thin wedges and served. The exterior of the cheese gets its spotted black and white colouring from the heat of the oven, and it’s Finnish name means “cheese bread” (since it’s baked like bread). The end product is a yummy cheese with a deliciously squeaky consistency. Tip: Leipäjuusto is often served with cloudberry jam as dessert, but the traditional Sámi way of eating it is to dip the wedges in hot, black coffee. Try it to experience the perfect blend of smooth, fatty cheese cut by hot, bitter coffee.; Mild but full of flavour, Finnish salmon soup: Finnish salmon soup is a classic that’s served both at home and in restaurants. The most popular version features creamy white broth studded with salmon, onions and potatoes and garnished with a handful of dill. You’ll find this delicacy on the menu at many restaurants and cafés – it’s a true comfort food that will warm you up on a winter’s day.; Endless varieties of new potatoes: New potatoes with herring (silli). New potatoes with fresh lake fish and chantarelle sauce. New potatoes with fish roe (mäti). New potatoes with just a knob of butter, some dill and a little salt. The variations of new potatoes available in Finland are seemingly endless and certainly mouth-watering. You’ll find that Finns can talk about new potatoes forever, as the little spuds hold the promise of summer. Look for them starting around midsummer and expect to see statistics on the harvest in all the local papers.; Poronkäristys from Lapland: Reindeer are found in the northern province of Lapland, and according to recent research, their meat is one of the healthiest you can put on your plate. It’s high in B-12, omega-3 and omega-6 – and it’s lean and delicious! Served alongside mashed potatoes, sautéed reindeer is a Finnish treat that’s eaten throughout the country, all year round.; Kalakukko – a local delicacy: Describing this local delicacy is difficult, but it’s essentially a combination of salty vendace and fatty pork that’s then wrapped in a rye crust and baked in foil. Best served alone or with heaps of butter, this delicacy is most delicious at its origin: a busy marketplace in Kuopio in Finland’s Lakeland. Loosely translated as “fish rooster,” this iconic pie’s name makes absolutely no sense, but it tastes incredibly good! And fun fact: the name of this unique delicacy has been granted protection under the EU quality scheme, along with Karjalanpiirakka, or Karelian pie.; Karjalanpiirakka – a sublime pastry: And speaking of Karjalanpiirakka, Karelian pie is the crown jewel of Finnish cuisine. This sublime pastry is originally from the eastern province of Karelia, and with its filling of delicious rice, potatoes or carrots enclosed in an incredibly crisp rye crust, this buttery delicacy will win you over in no time. It’s best served with plenty of fresh butter or a serving of egg butter-spread. Tip: You’ll find Karelian pies at any self-respecting Finnish marketplace, café, supermarket or gas station. Fresh, unbaked pies can also be purchased and baked at home!; Ruisleipä – made from sour dough: Ruisleipä – or rye bread – is a staple of the Finnish diet. While there are many varieties, the most popular and widely available is reikäleipä, meaning “bread with a hole.” People used to hang their bread on poles from the rafters, and while it’s dense, flat and very heavy, many Finns will actually have it sent through the post while they’re living abroad. Näkkileipä is the cracker version of rye bread and there are many varieties, including the internationally-sold Finn Crisp cracker. These are eaten at breakfast with butter, cheese and other spreads, with soups at lunch or even as an evening snack.; Rieska – Finnish flatbreads: Rieska is made with dough from a variety of local grains that’s then shaped into a chapati-like bread (or “kovaohranen”). If they’re baked with potato dough, they’re called “lepuska.” The magic of this flatbread lies in its freshness. Prepared on the same day they’re consumed, these savoury delights go great with heaps of butter and scorching black coffee. Tip: Try a different flatbread in each place you visit. Hopefully, you’ll have the chance to taste the local Savo variety, which is made with sour milk.; Korvapuusti – a pastry enjoyed with a cup of coffee: Korvapuusti translates to “slapped ears” in English, but these pastries are essentially cinnamon buns. And while Finland doesn’t hold a patent on cinnamon buns, it probably should. Usually enjoyed with a cup of coffee (Finns consume more coffee and, perhaps, more cinnamon buns than any other European nation), it can be difficult to stop at just one. Or two.; Bilberry pie – adored by all: In July and August, bilberries paint the Finnish forest. You’ll want to pick and freeze them for winter (like the Finns do), but these berries are best enjoyed in the summer months, whether on their own or in homemade pies. Although most Finnish berries can be made into pie filling, bilberry pie, or mustikkapiirakka, served with fresh milk, is one that’s adored by all.; Chocolate from Finland: Chocolate isn’t exactly a Finnish invention, but it’s prepared and sold all over the country. One place to enjoy it is at Brunberg, a famous chocolate shop in Porvoo. In operation since 1871, this family-owned and operated business offers a wide variety of tasty treats. Favourites include Truffles and Kisses, which have become hits among those who love high-quality chocolate. Fazer, another family-operated chocolate brand, can be found in any Finnish café, supermarket or kiosk – their products are even sold as souvenirs in the airport’s duty-free shop. The company produces the country’s most iconic chocolate, Fazerin Sininen, which is loosely translated and commonly known as “the Blue.” This chocolate, along with Fazer’s licorices and other sweets, are frequently shipped abroad, and their company-operated cafés are scattered throughout Finland. They serve Karelian pies and salmon soup, too!
        '''
    ),
    Document(
        page_content='''
        Guide to driving in Finland Here are some tips for driving in Finland – rain, snow or shine: While Finland’s roads are typically in good condition, Nordic weather occasionally throws drivers a curveball. Whether you encounter reindeer on the road in Lapland or ice…well, pretty much anywhere, here’s some practical information for a smoother, safer road trip.; Renting a car: There are car hire companies in all major towns and cities, as well as at airports. It’s worth booking in advance, and you must have a credit card on you when you book. If you have an EU or Swiss drivers’ licence, you can hit the road in Finland straight away.; Avoiding traffic: Traffic jams aren’t common in Finland. Although you might hear some complaints about traffic from Finns, it’s rare to hear them from those who live in big cities elsewhere. To a Finn, a few-minute delay is the equivalent of heavy traffic.; Speed limits: Generally, the speed limit in Finland is 50 km/h in built-up areas and 80 km/h outside of them. Both limits are typically enforced if there is no other speed limit posted. On major highways in summer, you can drive 100 km/h. On motorways, the summertime limit is 120 km/h. In winter, however, the speed limit on highways is reduced to 80 km/h, and on motorways, it’s 100 km/h.; Turn your lights on!: If an oncoming vehicle flashes its high beams at you, it usually means you don’t have your headlights on. It’s a law in Finland that drivers must have their lights on – no matter the season and even in the midnight sun. But being flashed could also mean there’s an accident ahead or an animal is on the roadway, so be sure to stay alert.; Paying (or not paying) tolls: There are zero toll roads or bridges in Finland. Yes, zero. That’s because constructing and maintaining roads and highways is mostly funded by Finnish taxes. So go on, enjoy the drive – it’s on us!; Traffic enforcement cameras: Finnish police use automatic traffic surveillance equipment on heavily trafficked roads and in urban areas. You’ll know you’re driving on one of these roads if you see a yellow road sign depicting a camera.; Beware of the animals: Drivers in Finland should stay alert for signs warning of elk crossing the roadways. If you see one, slow down, and at night, use high beams whenever possible. While many of the country’s major highways are lined with high elk fences, accidents do happen – and they can be fatal for drivers, passengers and animals. Also, if you’re driving in Lapland, a reindeer (or twelve!) on the road is a very common sight. Please make sure you drive carefully and adjust your speed to account for limited visibility and potentially dangerous driving conditions.; Essentials of driving in winter: The downside of the delightful winter weather in Finland is that, at times, roads can be very icy. Because of this, all vehicles must be equipped with winter tyres between December 1st and March 1st. You might even want to have studded tires if you’re driving in the northern part of the country. Also, be sure to set aside adequate time for driving in winter. When the roads are icy or there’s a heavy snowfall, the only way to go about it safely is to drive slowly. Note that roads are generally not salted. Instead, they’re cleared by snow ploughs. When driving an electric car, be aware that cold weather and cabin heating cut your driving power. And whether your car is powered by electricity or gasoline, it’s always a good idea to wear or bring warm clothes with you when driving in Finland – just in case you need to make an unexpected stop.
        '''
    ),
]

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True # Following tutorial specs
)
docs = text_splitter.split_documents(docs)

In [ ]:
docs

[Document(metadata={'start_index': 9}, page_content="10 new unique places to stay in Finland From modern wooden cabins and brand-new glass huts to resorts with a touch of luxury: Finland offers a plethora of unique accommodation options that showcase the country's breathtaking natural beauty both in the materials and the locations. In recent years, there has been upsurge in unique new hotels, resorts, and cabins opening up around the country. We've curated a list of the coolest and most recently opened ones where you can immerse yourself in Finnish nature and find your inner happiness. Article published in June 2023.; 1. Hilltop Forest – Inkoo, Helsinki region: Hilltop Forest, a small resort in Inkoo, offers charming triangular wooden huts with large windows facing a lush forest. Curl up in the fresh linen bedding and admire the view over the majestic pine trees and birches. The sole mission of the day is to stroll through the forest to the nearby spa. The resort's restaurant serves br

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore

# Initialise the FAISS retriever -> Ranked retrieval, Similarity search using Locality Sensitive Hashing with Random Projections
# https://python.langchain.com/docs/integrations/vectorstores/faiss/
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

index = faiss.IndexFlatL2(len(embeddings_model.embed_query("hello world"))) # Initialise FAISS index with the dimensionality

faiss_vector_store = FAISS(
    embedding_function=embeddings_model,
    index=index, # what index to use
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

faiss_vector_store.add_documents(docs)
retriever = faiss_vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 10}) # num docs to return from FAISS for re-ranking

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

**Before re-ranking**

In [ ]:
query = "Please provide me with the best accomodation in Finland"
docs = retriever.invoke(query)

In [ ]:
docs

[Document(metadata={'start_index': 9}, page_content="10 new unique places to stay in Finland From modern wooden cabins and brand-new glass huts to resorts with a touch of luxury: Finland offers a plethora of unique accommodation options that showcase the country's breathtaking natural beauty both in the materials and the locations. In recent years, there has been upsurge in unique new hotels, resorts, and cabins opening up around the country. We've curated a list of the coolest and most recently opened ones where you can immerse yourself in Finnish nature and find your inner happiness. Article published in June 2023.; 1. Hilltop Forest – Inkoo, Helsinki region: Hilltop Forest, a small resort in Inkoo, offers charming triangular wooden huts with large windows facing a lush forest. Curl up in the fresh linen bedding and admire the view over the majestic pine trees and birches. The sole mission of the day is to stroll through the forest to the nearby spa. The resort's restaurant serves br

Before re-ranking, we can see that second highest ranked document is about driving!

**After re-ranking**

In [ ]:
# Use Cross-Encoder
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-base")
compressor = CrossEncoderReranker(model=model, top_n=3) # top 3
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke("Please provide me with the best accomodation in Finland")
compressed_docs

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

[Document(metadata={'start_index': 9}, page_content="10 new unique places to stay in Finland From modern wooden cabins and brand-new glass huts to resorts with a touch of luxury: Finland offers a plethora of unique accommodation options that showcase the country's breathtaking natural beauty both in the materials and the locations. In recent years, there has been upsurge in unique new hotels, resorts, and cabins opening up around the country. We've curated a list of the coolest and most recently opened ones where you can immerse yourself in Finnish nature and find your inner happiness. Article published in June 2023.; 1. Hilltop Forest – Inkoo, Helsinki region: Hilltop Forest, a small resort in Inkoo, offers charming triangular wooden huts with large windows facing a lush forest. Curl up in the fresh linen bedding and admire the view over the majestic pine trees and birches. The sole mission of the day is to stroll through the forest to the nearby spa. The resort's restaurant serves br

After re-ranking, we can see that the top 3 documents are now all relevant to our query, demonstrating the usefulness of re-ranking i.e. how it supports retrieval

**Other re-rankers**

Refines initial retrieval results

Optimise RAG by providing LLMs with more user-relevant information

LLMs may suffer from context stuffing: Less is more. The recall performance for LLMs decreases as we add more context (increased context window)
- Rerank documents and feed the top few into the LLM
- https://www.pinecone.io/blog/why-use-retrieval-instead-of-larger-context/
- https://medium.aiplanet.com/advanced-rag-cohere-re-ranker-99acc941601c


<u>LLM Strategies</u>
- LLM analyses semantic relevance between query and each document. LLM assigns relevance scores, enabling the re-ordering of documents to prioritise the most relevant ones
- NVIDIA NeMo Retriever Reranking NIM
  - NVIDIA provides embeddings: ```NVIDIAEmbeddings()```
  - https://python.langchain.com/docs/integrations/text_embedding/nvidia_ai_endpoints/
  - The NVIDIA re-ranking model is optimised for providing a probability score that a given passage contains the information to answer a question. It re-ranks the prev fetched chunks according to which is most relevant using the same query
  - Combine multiple data sources to rerank: FAISS + BM25
  - Can put it in the RAG pipeline
  - https://developer.nvidia.com/blog/enhancing-rag-pipelines-with-re-ranking/
  - https://build.nvidia.com/nvidia/rerank-qa-mistral-4b?docker=false
- Cohere
  - Also uses LLMs
  - https://docs.cohere.com/docs/overview
  - https://medium.aiplanet.com/advanced-rag-cohere-re-ranker-99acc941601c
  - https://python.langchain.com/docs/integrations/retrievers/cohere-reranker/
  - https://js.langchain.com/docs/integrations/document_compressors/cohere_rerank/




**Both require API key**

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

## **Part 6: Generate**

<u>Overview of Generate Step</u>

An LLM produces an answer using a prompt that includes the question and retrieved data

A chain in LangChain is a sequence of interconnected components that process a user's query to generate and deliver valuable output. It is like a pipeline

The components of a chain typically include:

- Prompts: Templates that guide the LLM's responses.

- LLMs or Chat Models: The engines that generate responses based on the prompts.

- Output Parsers: Tools that parse the LLM's output.

- Tools: Extensions that allow LLMs to extract additional information from APIs or run code, turning LLMs into agents.

- General Functions: Additional functionalities that can be chained together.

In LangChain, these components, known as runnables, can be combined/chained to form a comprehensive pipeline. Pipelines are a way to think of chains

An LLM chain is similar to a data pipeline. In a data pipeline, raw data is transformed into clean, structured data.

Similarly, in an LLM chain, a query is transformed into valuable output using LLM calls, functions, and additional data.

<br/>

<u>More on Prompts</u>

A prompt for a language model is a set of instructions provided by a user to guide the model's response, helping it to understand the context and generate relevant and coherent output

Prompt templates in LangChain are predefined receipes for generating prompts for language models

```
from langchain.prompts import PromptTemplate

# Create a simple prompt template
# Formulate the prompt to the LLM with a string as a placeholder, in this case {topic}
prompt_template = """
You are a helpful assistant that explains AI topics. Given the following input:
{topic}
Provide an explanation of the given topic.
"""

# Create the prompt from the prompt template from LangChain
prompt = PromptTemplate(
    input_variables=["topic"],
    template=prompt_template,
)

# Assemble the chain using the pipe operator "|", more on that later. Assemble a chain using both components
chain = prompt | llm

# Invoke the chain with an input variable
chain.invoke({"topic":"What is LangChain"})
```

LangChain rag-prompts from prompt hub:
https://smith.langchain.com/hub/rlm/rag-prompt

<br/>

<u>LangChain Expression Language (LCEL)</u>

LangChain Expression Language (LCEL) simplifies building complex chains from basic components.

It uses the pipe operator (|) to chain different components, feeding the output from one component as input to the next.

 A simple example of a chain composed this way would be a prompt combined with a model and an output parser.

 ```chain = prompt | model | output_parser```

 These components are called "runnables". Think of LangChain Expression Language(LCEL) as a declarative way of composing these runnables into chains.

 We can use the string output of this chain and send it to a new chain that will return a different output

LangChain allows multiple chains to be chained together in this manner.

```chain = chain_1 | chain_2```

The Runnable Protocol

A runnable is a unit of work that can be invoked, batched, streamed, transformed, and composed.

The chains we build with LangChain and their components (the components of those chains) are runnables.

We can also pass arbitrary functions into a chain, which will be converted into runnables.

```chain = prompt | (lambda input: {"x":input}) | model | output_parser```

These chains can be:

```
chain.invoke(...)
chain.batch([...]) if we have multiple inputs
chain.stream(...)
```

Each of these components (```retriever, prompt, llm, etc.```) are instances of Runnable. This means that they implement the same methods - such as sync and async ```.invoke```, ```.stream```, or ```.batch``` - which makes them easier to connect together. They can be connected into a ```RunnableSequence```--another Runnable--, via the ```|``` operator.

LangChain will automatically cast certain objects to runnables when met with the | operator.

Core runnable objects in LangChain:

RunnableSequence: A class that chains together multiple runnable components, ensuring each component processes its input and sequentially passes its output to the next component in the pipeline.

RunnableLambda: A class that turns a Python callable (like a function) into a runnable component, allowing integration of arbitrary functions into chains.

RunnablePassthrough: A class that either passes its input through unchanged or adds additional keys to the output. It can act as a placeholder or allow flexible integrations into sequences where we need to modify the input.

RunnableParallel: A class that runs multiple runnables concurrently, allowing branching where two chains run on the same input but return different outputs (return a mapping of their outputs).

<br/>

<u>Experimentation and Improvements</u>
1. Citations (Must do)
2. Prompt Engineering?



In [ ]:
from langchain import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Question: {question}
Context: {context}
Helpful Answer:
"""

prompt = PromptTemplate.from_template(template)

In [ ]:
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="\nYou are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nQuestion: {question}\nContext: {context}\nHelpful Answer:\n")

In [ ]:
output_parser = StrOutputParser()

chain = (
    {"context": (lambda x: x["question"]) | retriever,

     "question": (lambda x: x["question"])}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
answer = chain.invoke({"question":"What are the best things to do in Norway?"})

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [ ]:
print(answer)


You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Question: What are the best things to do in Norway?
Context: [Document(metadata={'source': 'Norway Example.txt', 'start_index': 1786}, page_content="9. Mount Hanguren, Sognefjord/ Hardangerfjord\n\n10. Mount Fløyen, Sognefjord/ Hardangerfjord\n\n11. Mount Ulriken, Sognefjord/ Hardangerfjord\n\n12. Rimstigen, Næroyfjord\n\n13. Urkeegga, Hjørundfjord\n\n14. Himakånå, Nedstrandsfjorden\n\n15. Langfoss Waterfall Hike, Åkrafjord\n\nWe've suggested some day hikes within the fjords, but given that wild camping is permissible in Norway we also recommend embarking on a multi-day adventure.\n\nPlease note that the route descriptions we have included are summaries, designed to inspire rather than for any navigational purpose. Further mapping and planning - or an experienced local guide - will be needed to safely walk 

The best things to do in Norway include hiking in the fjords. Some of the best fjords for hiking are Nærøyfjord, Hjørundfjord, Sognefjord, and Hardangerfjord. These fjords offer excellent summit hikes with stunning views. Some popular hikes include Preikestolen, Breiskrednosi Summit Hike, Romsdalseggen Ridge, Mount Skåla, Trolltunga, Aurlandsdalen Valley, Dronningstien, Vidasethovden, Mount Hanguren, Mount Fløyen, Mount Ulriken, Rimstigen, Urkeegga, Himakånå, and Langfoss Waterfall Hike. These hikes offer a range of difficulties and are accessible by public transport. The best time to visit Norway's fjords for hiking is from spring to October, with peak season being mid-June to the end of July. However, weather can be changeable, so it's important to pack accordingly.

In [ ]:
chain_skipprompt = (
    {"context": (lambda x: x["question"]) | retriever,

     "question": (lambda x: x["question"])}
    | prompt
    | llm.bind(skip_prompt=True)
    | StrOutputParser()
)

In [ ]:
answer = chain_skipprompt.invoke({"question":"What are the best things to do in Norway?"})

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [ ]:
print(answer) # Check why is this repeated for some LLMs?

The best things to do in Norway include hiking in the fjords. Some of the best fjords for hiking are Nærøyfjord, Hjørundfjord, Sognefjord, and Hardangerfjord. These fjords offer excellent summit hikes with stunning views. Some popular hikes include Preikestolen, Breiskrednosi Summit Hike, Romsdalseggen Ridge, Mount Skåla, Trolltunga, Aurlandsdalen Valley, Dronningstien, Vidasethovden, Mount Hanguren, Mount Fløyen, Mount Ulriken, Rimstigen, Urkeegga, Himakånå, and Langfoss Waterfall Hike. These hikes offer a range of difficulties and are accessible by public transport. The best time to visit Norway's fjords for hiking is from spring to October, with peak season being mid-June to the end of July. However, weather can be changeable, so it's important to pack accordingly.


Comparing it to no-RAG LLM response

In [ ]:
print(llm.invoke("What are the best things to do in Norway??"))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


What are the best things to do in Norway???

Norway is a country known for its stunning natural beauty, rich history, and vibrant culture. Here are some of the best things to do in Norway:

1. **Visit the Fjords**: Norway's fjords are one of its most iconic natural features. The most famous is the Geirangerfjord, but others like the Hardangerfjord and the Sognefjord are also breathtaking. You can explore them by boat, kayak, or even hike along their edges.

2. **Explore Bergen**: This historic city is a gateway to the fjords and offers a mix of cultural attractions, museums, and vibrant neighborhoods. Don't miss the Bryggen Hanseatic Wharf, a UNESCO World Heritage site.

3. **Experience the Northern Lights (Aurora Borealis)**: If you're lucky enough to visit Norway between September and March, you might get to see this natural light show. The best places to view them are in the north, such as Tromsø or the Lofoten Islands.

4. **Visit Oslo**: Norway's capital offers a blend of history,